In [1]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage # The foundational class for all message types in LangGraph
from langchain_core.messages import ToolMessage # Passes data back to LLM after it calls a tool such as the content and the tool_call_id
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage # Message for providing instructions to the LLM
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END,START
from langgraph.prebuilt import ToolNode,create_react_agent

import os
# from langgraph.prebuilt import ToolExecutor
from langchain.agents import create_openai_functions_agent, AgentExecutor

import requests
from bs4 import BeautifulSoup
load_dotenv()

True

In [2]:
from tools.pull_businesstoday import get_topic_news

In [3]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [4]:

model = ChatOpenAI(model = "gpt-4o")

In [5]:
def stock_summarizer(state:AgentState) -> AgentState:
    """
    stock recent news provider agent 
    """
    system_prompt = SystemMessage(
        content="""
        You are a financial news summarizer working with web-scraped content from known news websites (e.g., Moneycontrol, Business Today).
        Your task is to summarise the following news article(s) in a clear, concise, and fact-preserving way. Focus only on the essential information that would be relevant for financial markets, investors, or traders directly or indirectly .
        DO:
        - Extract and summarise actual financial and business news from each section
        - Retain as much factual detail as possible: company names, numbers, dates, events, and announcements
        - Write clearly and concisely but do **not omit important details** for the sake of brevity
        - Include multiple companies, events, or sectors if mentioned
        - Ignore only UI elements, ads, or obvious web clutter — everything else should be preserved
        - always add the current  timestamp in the header
        - Use to tool to get stock news and summarize
        DO NOT:
        - Perform sentiment analysis
        - Classify or extract entities
        - Infer or hallucinate missing details
        """
    )
    tools = [get_topic_news]
    

    agent_graph = create_react_agent(
    model=model,
    prompt=system_prompt,
    tools=tools)
    input = {"messages":[{"role": "user","content":"You are a financial news summarizer working with web-scraped content from known news websites (e.g., Moneycontrol, Business Today)."}]}
    response = agent_graph.invoke(input)

    return response

In [6]:
def stock_picker(state:AgentState)->AgentState:
    """
    Stock picker agent 
    """
    system_prompt = SystemMessage(
        content="""You are a stock trader and suggest good and bad stocks as per the news summary from the previous agent 
      timestamp": "<current IST time>,
      recommendations: [
        {"entity": "RELIANCE", "action": "BUY", "reason": "..."},
        {"entity": "INFOSYS", "action": "BUY", "reason": "..."},
        {"entity": "Banking sector", "action": "HOLD", "reason": "..."}
        ]
        """
    )    
    response = model.invoke([system_prompt] + state["messages"])
    return {"messages": [response]}
    

In [7]:
workflow = StateGraph(AgentState)

In [8]:
workflow.add_node("summerizer",stock_summarizer)
workflow.add_node("trader",stock_picker)

workflow.set_entry_point("summerizer")
workflow.add_edge("summerizer","trader")
workflow.add_edge("trader",END)

In [9]:
app=workflow.compile()

In [10]:
# app

In [11]:
prompts="""
As per the current news tell me some good sector and stocks i can invest  with reason 
"""

inputs = {"messages": [(f"user,{prompts}")]}


response=app.invoke(inputs)

In [12]:
# response={'messages': [HumanMessage(content='You are a financial news summarizer working with web-scraped content from known news websites (e.g., Moneycontrol, Business Today).', additional_kwargs={}, response_metadata={}, id='4f1af970-75af-4a70-9dae-0f735237cb76'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lQtAX0I2KBlN1mlfQ2hXd2Ub', 'function': {'arguments': '{}', 'name': 'get_topic_news'}, 'type': 'function'}, {'id': 'call_BCj4zYpikOre51Qgg29tdRVK', 'function': {'arguments': '{}', 'name': 'get_topic_news'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 448, 'total_tokens': 489, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bd18a5f5-d645-4d16-bc4f-cbe41900e2f5-0', tool_calls=[{'name': 'get_topic_news', 'args': {}, 'id': 'call_lQtAX0I2KBlN1mlfQ2hXd2Ub', 'type': 'tool_call'}, {'name': 'get_topic_news', 'args': {}, 'id': 'call_BCj4zYpikOre51Qgg29tdRVK', 'type': 'tool_call'}], usage_metadata={'input_tokens': 448, 'output_tokens': 41, 'total_tokens': 489, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content='[\'new_source_bankingnews\', \'heading: Federal Bank’s corporate loan book to grow 8-10% in FY26 on tariffs uncertainty, cheaper funds at bond market: CEO\', \'content:According to the investor presentation of the bank, corporate loan book of the bank increased by 6 percent on-year to Rs 83,680 crore in Q1FY26, from Rs 79,208 crore in a year ago period, and Rs 83,680 crore in a quarter ago period.\', \'heading: Federal Bank Q1 results: Net profit falls 15% on-year to Rs 862 crore\', \'content:The bank registered operating profit of Rs 1556.29 crore for the quarter ended June 30, 2025.\', \'heading: Financial bids for IDBI Bank stake by December quarter, winning bid likely by end of fiscal: DIPAM Secretary\', \'content:The government has completed the process of due diligence, and data room protocols have been completed for all interested parties, the secretary said, adding that formal consultations have been completed with all qualified and interested parties.\', \'heading: ICICI Lombard scaled down motor, fire insurance coverage over pricing pressure: CFO Gopal Balachandran\', \'content:ICICI Lombard CFO Balachandran told Moneycontrol growing those segments may not make financial sense, citing stagnant third-party premium rates over the past 4-5 years.\', \'heading: PSU banks dominated bond issuances in FY25, more Tier-II debt raise likely this fiscal\', \'content:Private lenders have been totally absent from hybrid bond issuances in FY25. Despite favorable conditions for Tier-2 debt, these banks chose to rely on internal accruals and equity-driven capital planning.\', \'heading: Distributors opting for short-term health plans over long term post commission rule change: Niva Bupa CEO\', \'content:Krishnan Ramachandran, CEO, says the health insurer’s internal goal is to grow 5-10 percentage points faster than the overall health insurance industry\', \'heading: If I were Allianz, I’d be more worried about retaining Go Digit than the other way around: Chairman Kamesh Goyal\\xa0\', \'content:Addressing concerns over the Allianz-Jio reinsurance JV, Goyal said Allianz intends to continue its partnership with Go Digit, and the existing contract allows no mid-term changes, Goyal says\', \'heading: IDFC First shareholders grant Warburg affiliate board seat, reversing earlier decision\', \'content:While Currant Sea’s board nominee has not yet been disclosed, this marks a significant change in shareholder sentiment from May 2025, when a similar resolution was voted down.\', \'heading: RBI’s fraud detection tool MuleHunter AI expands reach with over 15 more banks nearing rollout\', \'content:The banks that have implemented the MuleHunter AI are Canara Bank, Punjab National Bank, Bank of India, Bank of Baroda, and AU Small Finance Bank, Suvendu Pati added\', \'heading: Indian rupee opens 2 paise higher at 87.58/USD on lingering worries over Trump tariff\', \'content:The local currency opened at 87.58 against the US dollar as compared to 87.60 against the greenback at previous close. According to the Bloomberg data, the domestic currency has depreciated 1.70 percent in July.\', \'heading: Fed’s hard line may force the RBI to keep purse wide open\', \'content:The Monetary Policy Committee (MPC) can’t afford to sit on the fence—it’s time for bold monetary stimulus to support growth.\', \'heading: Loan against gold jewellery grew 124% in June, fastest among personal loans, RBI data shows\', \'content:In absolute terms, the outstanding loan against gold jewellery stood at Rs 2.77 lakh crore as on June 27 . There has been a decline in the consumer durable segment\', "heading: DCB Bank\'s June quarter net profit rises 20% on year at Rs 157 crore", \'content:The Gross NPA as on June 30, 2025 was at 2.98 percent, while Net NPA was at 1.22 percent. The Provision Coverage Ratio (PCR) came in at 74.04 percent and PCR without considering gold loan NPAs was at 75 percent.\', \'heading: PSU banks hit harder by compression on yield on advances in Q1FY26\', \'content:According to data compiled by Moneycontrol, yield on advances for PSU banks dropped by 15–71 basis points (bps), significantly more than the 12–30 bps reduction observed for private banks.\', \'heading: Indian bond yields open 2 bps higher as Trump announces 25% tariffs\', "content:US Fed\'s Federal Open Market Committee decided to keep interest rates unchanged at 4.25 percent to 4.5 percent in the July 29-30 meeting", \'heading: Rupee slides further, opens 28 paise down at 87.71 after Trump slaps 25% tariff, Russia penalty\', \'content:The rupee has depreciated 0.83% in the last two weeks. This year, so far, the currency is the worst performing among Asian peers, down 3.25%\', \'heading: Indian rupee may open close to 88-mark against US dollar as Trump announces 25% tariffs on India\', \'content:According to the Bloomberg data, Indian rupee has depreciated 0.83 percent in last two weeks. On a year-to-date basis, the local currency is under pressure and second worst performing among Asian peers. It is down 3.25 percent, Bloomberg data showed.\', \'heading: Rupee at over at four-month low at 87.43 against dollar on Trump’s tariff threat\', \'content:The Rupee ended at 87.4250 against the US dollar compared to 86.8200 against the greenback at previous close, lower by 61 paise down on July 30.\', \'heading: Capital adequacy of PSU banks outpaces that of private lenders amid fundraising efforts\', \'content:Analysts expect capital buffers to remain strong across the banking sector. ICRA projects a 40–50 basis point increase in Tier-I capital ratios for PSBs this fiscal\', \'heading: PSU lenders’ average valuation surged 225% over five years on improved financials and asset quality\', \'content:According to data compiled from 11 major PSU banks, their average price-to-book (P/B) ratio has jumped from 0.35 in March 2020 to 1.14 in July 2025.\', \'heading: Merchant bankers tasked to bring foreign capital into PSU banks during upcoming QIPs\', \'content:The government budgets to earn approximately Rs 20,000 crore through QIP of five banks, namely Bank of Maharashtra, Indian Overseas Bank, UCO Bank, Central Bank of India, and Punjab and Sind Bank.\', "heading: Rupee falls to weakest since mid-March fearing impact of Trump\'s tariff on exports", \'content:The weakness is likely due to worries over the potential impact of the tariff rate on exports, though Reuters reported that there has likely been an intervention by the Reserve Bank of India that has helped limit losses.\', \'heading: Chart of the Day | India’s digital payments are surging, so are the risks\', \'content:As India’s digital payments soar to new heights, the journey to a cashless economy faces critical tests in security, inclusion, and regulation.\', \'heading: MPC Poll | RBI may hold rates in August policy to assess new data on inflation\', \'content:The central bank, so far has reduced the repo or the benchmark rate by 100 basis points (Bps) since February to aid growth, with 25 bps each in February and April respectively, and 50 bps in the June policy.\', \'heading: Not in a hurry\\xa0to burn our fingers, says CEO Aggarwal after Shriram General gives crop tender bids a miss\', \'content:Anil Kumar Aggarwal says the insurer is targeting a full-year premium of Rs 4,600 crore and could reach Rs 5,000 crore if demand picks up during the upcoming festival season\', \'heading: Rupee opens 18 paise down at 86.84 as dollar index surges\', "content:The dollar index, which measures the greenback\'s strength against a basket of six major currencies, rose to 98.64 in the early morning trade from below-97 levels in the previous session", \'heading: UPI’s free ride for all nearing its end?\', \'content:As RBI Governor Sanjay Malhotra signals the end of free UPI transactions, the move could reshape India’s fintech revolution, balancing innovation with sustainability\', \'heading: IndusInd Bank chairman assures investors about CEO hunt\', \'content:Chairman Mehta says, the bank has followed due process and timelines in submitting its recommendations, which remain consistent with its original proposal\', \'heading: IndusInd Bank sees further scope to cut savings, term deposit rates, says Soumitra Sen\', \'content:The cost of deposit of the bank in the reporting quarter eased by 9 basis points (Bps) year-on-year, and 6 bps on-quarter.\', \'heading: Why IRDAI wants insurers to appoint an in-house ombudsman and what it means for policyholders\', \'content:The insurance regulator has released a draft framework to mandate internal ombudsmen within insurance companies, and is expected to finalise the regulations by Q3FY26. Moneycontrol explains everything you need to know.\', \'heading: Double Trouble: IT layoffs & loan slowdown – is there a storm brewing?\', \'content:Private banks, which have higher exposure to private sector employees, particularly those from IT, have witnessed an 8-12 percent contraction in their FY26 EPS estimates.\', \'heading: MC Interview| Canara Bank to transfer 8 NPAs worth Rs 4,000 cr to NARCL by Q3: CEO K. Satyanarayan Raju\', \'content:Canara Bank’s total SMA 2 book was Rs 6,856 crore, of which Rs 5,000 crore pertain to two accounts—Rs 3,000 crore guaranteed by a state government for an irrigation project, and a rental discount real estate project in Bengaluru, Raju said.\', "heading: Aim to be among Top 3 players in India, says UBS\' Abhishek Joshi", "content:Betting on UBS and Credit Suisse\'s synergies,  Abhishek Joshi, Head of ECM India, UBS, is confident that the Indian market will remain attractive from an IPO and deals perspective this year", \'heading: Shriram Finance looks at new borrowing strategy to reduce costs\', \'content:Early onset of monsoon led to delay in construction activities to August and September, which had resulted in fall in AUM on construction equipment in Q1FY26, Revankar said.\', \'heading: Kotak Mahindra Bank Q1 FY26: a blip not a blow\', \'content:The bank had an unpleasant start to FY26\', \'heading: Rupee opens 5 paise up at 86.47 dollar as dollar index eases\', \'content:In the near term, the rupee is likely to trade in a the 86–87 range, experts say\', "heading: Banking Central | How to tackle rising frauds in India\'s banking sector", \'content:With frauds soaring to Rs 36,014 crore in 2024-25, the RBI’s latest data paints a grim picture of systemic cracks. Time for tougher oversight and accountability.\', \'heading: Kotak Mahindra Bank flags stress in vehicle loans, resumes microlending cautiously\', \'content:Bank sees MFI stress peaking, cautiously restarts disbursements across key States\', \'heading: Kotak Mahindra Bank Q1 Results: Net profit declines 7% to Rs 3,282 crore\', \'content:Provisions surge 109% yoy to Rs 1,208 crore; NII up 6% to Rs 7,259 crore\', \'heading: Zubin Mody resigns as CHRO of IndusInd Bank, cites pursuit of new opportunities\', \'content:In his resignation letter, Zubin Mody expressed that stepping down from the Bank was a difficult decision, marking the end of a fulfilling two-decade-long journey\', \'heading: DFS Secretary urge insurance brokers to help increase coverage in rural India, smaller cities\', \'content:Insurance brokers can augment insurance penetration through grassroots engagement, simplified product dissemination, and capacity building of local intermediaries, Nagaraju said.\', "heading: IRDAI\'s Satyajit Tripathy cautions on practices of insurance brokers to increase valuation", \'content:ripathy said that in the insurance broking space, there is a record level of activity in terms of mergers, acquisitions, and demergers in the distribution segment. “The regulator is witnessing increased activity in this area on a day-to-day basis,” he said.\', \'heading: Bank of Baroda Q1 net profit rises 1.9% on-year to Rs 4,541 crore\', "content:During the June quarter, BoB\'s total interest income rose 4.9 percent on-year to Rs 31,091 crore compared to Rs 29,629 crore a year ago.", \'heading: Vault Matters | Why now isn’t the time to upgrade banks\', \'content:While there are at least four bank requests pending for regulator’s blessings to get a bump up in the value chain, now is the time to observe and assess, not react and regret\\xa0\', \'heading: RBI to consolidate 8,000 regulations, set up review cell, says governor Sanjay Malhotra\', \'content:The move is aimed as simplifying compliance and enhancing clarity. The review cell will revisit all regulations every five–seven years to weed out obsolete norms and examine if there’s the right balance between efficiency and financial stability\', \'heading: Indian rupee opens 16 paise lower at 86.57/USD on dip in Asian currencies, eye on FOMC next week\', \'content:Asian currencies such as South Korean Won was down 0.27 percent from previous close, Philippine Peso was down 0.26 percent, Japanese Yen and Indonesia Rupiah was down 0.12 percent each, and Thai Baht was down 0.05 percent, as per Bloomberg data.\', \'heading: Bajaj Finance’s 2 & 3-wheeler, MSME businesses to grow slow in FY26: MD Rajeev Jain\', \'content:Two & Three-Wheeler Finance AUM declined by 20 percent on-year to Rs 15,703 crore in April-June quarter, from Rs 19,689 crore in a year ago period.\', \'heading: Bajaj Finance to submit detailed succession planning in six months to board: MD Rajeev Jain\', \'content:While deliberations on succession plan would start in 6 - 8 months, Jain to remain at the helm of affairs till March 2028.\', \'heading: Bajaj Finance curtails Karnataka exposure by up to 50%, amid political, regulatory risk: MD Rajeev Jain\', \'content:Karnataka accounted for 11 percent of the company’s balance sheet, making this a significant recalibration\', \'heading: Bajaj Finance reports Rs 1,556 crore microfinance loans in Q1FY26 after separating MFI from Rural B2C Portfolio\', \'content:In April-June quarter, Bajaj Finance has separated microfinance business from the Rural B2C loans. The company is sourcing MFI business through MFI branches.\', \'heading:Watch\', \'heading:You got 30 Day’s Trial of \', \'new_source_todaymarket\', \'heading:COMPANIES\', \'heading:NEWS\', \'heading:DCB Bank Q1FY26 | Net Profit Up 20% At Rs 157 Cr | Top Management Excl\', "content:DCB Bank\'s standalone net profit jumped 19.71% to Rs 157.26 crore in Q1 FY26, compared with Rs 131.36 crore in Q1 FY25. Total income increased 25.57% to Rs 2,049.69 crore in Q1 FY26 as against Rs 1,632.25 crore in Q1 FY25. Net interest income (NII) climbed 16.90% to Rs 581 crore during the quarter, compared with Rs 497 crore posted in corresponding quarter last year. Deposits stood at Rs 62,039 crore as of 30 June 2025, registering the growth of 20.02%, compared with Rs 51,690 crore as of 30 June 2024. Net advances grew 21.41% year on year to Rs 51,215 crore in Q1 FY26. As of 30 June 2025, CASA ratio was at 23.32% as against 25.41% as of 30 June 2024. Gross non-performing assets (NPA) stood at Rs 1,553.63 crore as on 30 June 2025 as compared with Rs 1,353.47 crore as on 30 June 2024. Catch Shailendra Bhatnagar with Q1 earnings conversation with Praveen Kutty, MD & CEO, DCB Bank decoding the bank results", "heading:Tuhin Kanta Pandey: SEBI To Enforce \'@valid\' UPI For Safer Market Transactions From Oct 1", \'content:SEBI Chairman Tuhin Kanta Pandey, speaking at an ICAI event, highlighted a key investor protection measure being operationalised by SEBI. From October 1, 2025, financial transactions with SEBI-registered intermediaries via UPI must be routed through a verified domain—“@valid”—developed in coordination with NPCI. While the initiative has been under development, it is now being formally rolled out to raise awareness and enhance transaction security in the securities market. The “@valid” UPI handle will help investors verify the authenticity of intermediaries and reduce the risk of fraud, ensuring secure, regulated payment channels across all UPI-linked investment platforms. \\r\\n\', \'heading:US President Donald Trump Taunts India, Backs Pakistan Pact Amid Tariff Row\', \'content:Donald Trump’s latest comments have stirred controversy as he hints at a trade pact with Pakistan while criticizing India’s ties with Russia. Trump slammed India’s “high tariffs” and dismissed prospects of stronger trade relations, claiming the US has “done very little business” with India. Mr. Sabnavis reacts, calling it a ploy under Trump’s “Make America Great Again” agenda – aimed at rattling trade partners and gaining leverage. Despite these remarks, Sabnavis asserts India’s strong sovereign stance and ability to protect its economic interests. As the geopolitical drama unfolds, questions rise on how India will navigate trade talks with the US while safeguarding strategic autonomy and export competitiveness amid a volatile global order. \', \'heading:Will Tata Motors Slip Below ₹600? Deven Choksey On $4B Iveco Deal\', \'content:Tata Motors’ $4 billion IVECO deal is set to reshape its global ambitions. Deven Choksey explains how this transaction could expand Tata Motors’ commercial vehicle business to over ₹2 lakh crore in size, tripling its scale. The deal brings advanced technology and a stronger global footprint, boosting long-term growth prospects. While short-term market adjustments may occur, Choksey believes this is a big opportunity for Tata Motors as its passenger and commercial vehicle divisions undergo demerger. Will the stock hold above ₹600, and what does it mean for EPS? Watch this detailed breakdown of the deal’s impact on Tata Motors’ global strategy, profitability, and investor outlook. \', "heading:Deven Choksey Explains Jio\'s ₹15,000 Cr Fund-Raise Move and Insurance Gameplan", \'content:Reliance Jio has announced the allocation of 50 crore warrants at ₹316.50 to the promoter family, raising nearly ₹15,000 crore. But why is Jio raising such a big amount? Market expert Devan Choksey breaks down the reason behind this move and what it means for Jio’s future plans. The funds are likely to be deployed for Jio’s ambitious ventures in financial services, including its joint reinsurance business with Allianz. Reinsurance is a highly capital-intensive sector, and this investment could make Jio-Allianz one of India’s largest players in the space. Watch as Devan Choksey explains the strategy, growth potential, and long-term impact of this stake hike on Reliance Jio’s expansion plans. \', \'heading:KPIT Tech Q1 Result: Strong Deal Wins Of $241 Mn | Top Management Decodes The Growth, Outlook & More\', \'content:KPIT technologies Q1 Results: Reported its Q1 FY26 results with revenue at ₹1,538.8 crore, up 12.8% YoY, while net profit fell to ₹171.9 crore. The company retained its FY26 guidance of 18–22% revenue growth in constant currency and an EBITDA margin target of ~20.5%. Q1 saw strong deal wins worth US\\u202f$241 million TCV, largely in powertrain and connected vehicle projects across the US and Europe. Despite delays in execution timelines, the overall deal pipeline expanded ~20% YoY, reflecting strong client traction. The stock gained around 4% on the positive outlook and stable margins. Catch Shailendra Bhatnagar in conversation with Sachin Tikekar, JT MD & Co-founder, KPIT Technologies and Priya Hardikar, CFO, KPIT Technologies decoding Q1 results, outlook and more\', \'heading:Trump’s 25% Tariffs Could Hurt India’s Export Edge\', \'content:In this exclusive interview with Business Today TV, Ranen Banerjee, Partner and Economic Advisory Leader at PwC, discusses India’s tariff landscape and macroeconomic outlook. He notes that tariff uncertainty is reducing, but steep duties of 20–25% could still challenge India’s export competitiveness. While the pharma sector remains tightly coupled, other industries have room to diversify trade with the US. He speaks on the RBI’s neutral stance, subdued private investment, food inflation trends, and the rising cost of oil imports due to the Russia situation. Banerjee also underlines the need for continued government CAPEX to address high logistics costs and fuel long-term growth. \\r\\n\', \'heading:Tata Motors Slips 4% On Reports Of $4.5 Billion Iveco Deal\', "content:Tata Motors is reportedly considering a $4.5 billion bid to acquire Italian truck maker Iveco, in what could be the company’s biggest overseas deal since Jaguar Land Rover. We discussed the implications of this potential acquisition with Sharad Avasthi, Head of Research (PCG) at SMIFS. He shared insights on whether this could be Tata Motors\' next JLR moment, the possible benefits of global CV technology, and the strategic value of entering the premium truck segment. With Iveco already doubling in value this year and Tata Motors stock under pressure, we decode what this move could mean for shareholders, global expansion, and the future of the commercial vehicle business. ", \'heading:BOI Q1 Results: Profit Surges 32%, Stock Up 3% | CEO Rajneesh Karnatak On Strong Q1 & Outlook\', \'content:In this special episode of Easynomics on Business Today TV, Group Editor Siddharth Zarabi speaks with Professor Bharat Bhasker, Director of IIM Ahmedabad, about a milestone in Indian management education—the launch of IIM Ahmedabad’s first international campus in Dubai this September. Professor Bhasker shares why Dubai was chosen, the strategic focus on executive education, and how the new campus positions IIMA as a bridge to the Global South. The conversation delves into case-based pedagogy, AI-driven immersive learning, growing classroom diversity, and the institute’s response to a rapidly changing global business landscape. From fee structures and student intake to future expansion plans, this in-depth discussion offers rare insight into how India’s premier B-school is going global while staying true to its core mission of impact-led leadership. \', \'heading:Asian Paints Q1 Results: Top Management Decodes Expansion Plans, Growth Chart, Competition & More\', \'content:Asian Paints on Tuesday reported a 6% year-on-year (YoY) decline in its consolidated net profit for the June quarter at Rs 1,100 crore, compared to Rs 1,170 crore in the year-ago period. Revenue from operations marginally decreased by 0.20% YoY to Rs 8,924 crore. However, the company experienced a significant 59% sequential increase in PAT and a 7% QoQ rise in revenue compared to the previous quarter. Home Decor categories declined during the quarter with pressure on household disposable incomes. The International business saw value growth of 8.4% during the June quarter. Catch Earnings conversation with Shailendra Bhatnagar in conversation with Amit Syngle, MD & CEO, Asian Paints decoding the Q1 results, expansion plans and International Business growth \', "heading:InCred\'s Top Healthcare & Pharma Picks For The Next Decade", \'content:In this episode of Market Guru, Aditya Khemka, Fund Manager at InCred AMC, shares his top three high-conviction bets in the healthcare and pharma space for the next decade. He explains why Healthcare Global (HCG) is his preferred pick in the hospital segment, citing its strong focus on oncology and long-term patient retention. In the CDMO/API space, he highlights Jubilant Pharma for its niche offerings, high entry barriers, and solid execution track record. Lastly, he names FDC Limited as a standout in branded generics, thanks to its well-established household brands and potential for exponential growth. Watch as Khemka breaks down each company’s strengths and long-term value proposition for investors. \', \'heading:Asian Paints Q1 Results: Why Analyst Are Cautious On Asian Paints Despite Strong Q1 Sales Volumes\', \'content:Asian Paints has reported better-than-expected Q1 numbers with a 3.9% volume growth, surpassing market estimates of around 2%. The management commentary was also upbeat, hinting at stabilizing market share and improving industry momentum. However, despite these positives, Sudip Bandyopadhyay, Group Chairman, Inditrade Capital analyst remains cautious on taking fresh positions in the stock, citing rising competition from large conglomerates like JSW and Birla Opus and ongoing industry consolidation. Listen in \', \'heading:Motherson Sumi Wiring Q1 Results | CFO & COO On Profit Dip, EV Growth & Expansion Plans\', \'content:Business Today speaks to Gulshan Pahuja, Chief Financial Officer, and Anurag Gahlot, Chief Operating Officer of Motherson Sumi Wiring India Ltd (MSWIL), to discuss the company’s Q1 FY26 performance. Despite a 4% year-on-year decline in net profit to ₹143 crore, the company reported strong revenue growth of 14.2% to ₹2,494 crore, driven by new vehicle model launches and increased content per vehicle. The management addresses the reasons behind the dip in operating margins, which narrowed to 9.8% due to ramp-up costs at Greenfield projects. They also share insights into MSWIL’s growing electric vehicle (EV) segment, which now contributes 5.4% of total revenue, and reiterate their commitment to future-ready capacity expansion in alignment with OEM demand. The discussion also highlights the company’s continued debt-free status and its strategic focus areas for the upcoming quarters. \', \'heading:Aditya Khemka Of InCred AMC Creates Wealth For Investors With Pharma Small Caps & Patient Stock Pick\', \'content:On this episode of Market Guru by Business Today, we are joined by Aditya Khemka, Fund Manager at InCred Asset Management, who offers a rare and incisive look into his high-performing ₹550 crore healthcare portfolio. Known for his deep expertise in the pharma and diagnostics space, Mr Khemka explains what is fuelling the ongoing bull run in pharma stocks, and why hospital chains and branded generics form the core of his concentrated, small-cap-heavy strategy. In a wide-ranging conversation, he addresses concerns about pricing power in the hospital sector, the impact of potential US tariffs on Indian pharma exports, and why companies like Healthcare Global, Jubilant Pharma, and FDC are his top long-term bets. Mr Khemka also shares valuable insights on market valuations, sectoral trends, and his approach to bottom-up investing. An unmissable session for serious investors looking to understand the future of Indian healthcare investing.\', \'heading:Waaree Energies Q1 Results: Profit Zooms 92% YoY | Top Mgmt Decodes Q1 Highlights & Growth Plan\', \'content:Waaree Energies Q1 Results: The company posted a quarterly revenue of ₹4,597.18 crore, marking a 31.48% YoY growth, with EBITDA at ₹1,168.67 crore (up 82.61% YoY) and margins at 25.42%. Profit After Tax stood at ₹772.89 crore, a sharp 92.68% YoY rise. Key developments include a proposal to change the IPO object clause for shifting manufacturing from Odisha to Gujarat and Maharashtra, commissioning plans for additional module manufacturing capacities of 1.6 GW in Texas (USA) and 3.2 GW in Chikhli (Gujarat), and ongoing construction of green hydrogen, inverter, and battery energy storage facilities. Waaree Solar Americas Inc. recorded an order inflow of ~2.23 GW. The board also approved an additional capex of ₹2,754 crore for expanding cell capacity by 4 GW in Gujarat and ingot-wafer capacity by 4 GW in Maharashtra. Catch Sakshi Batra in conversation with Dr. Amit Paithankar, CEO & Whole-time Director, Waaree Energies Limited decoding the Q1 results and growth plan .\', \'heading:Daily Calls LIVE: Ask Your STOCK MARKET TODAY QUERIES | Market Update LIVE | Share Market News Today\', "content:BTTV brings you a new market show - \'Daily Calls,\' where you can gain invaluable insights and clarity on your market queries through our live sessions featuring expert analysts. Whether you\'re confused about where to invest, how to invest, or how to build and structure your portfolio.", \'heading:Maharashtra Temples & Trusts Can Now Invest In Stock Markets, Mutual Funds! Check Details\', "content:Maharashtra\'s iconic temples and charitable trusts could soon become active investors in India’s financial markets. With a sweeping reform effective July 21, 2025, Maharashtra state government has opened the gates for public trusts to allocate up to 50% of their corpus in mutual funds and market-linked securities. For decades, public trusts—be it religious institutions, social welfare organizations, or educational charities—were largely confined to traditional instruments such as fixed deposits and post office schemes. Any deviation into riskier assets required cumbersome, case-by-case permissions from the Charity Commissioner but that changes now. Catch Sakshi Batra explaining this mega move.", \'heading:TCI Q1 Results: Net Profit Rise 17% | Top Management Decodes The Growth Print For FY26\', "content:TCI Q1 Results: Transport Corporation of India Ltd (TCI) reported a consolidated net profit of ₹107.2 crore for the quarter ended June 30, 2025. The company\'s revenue from operations stood at ₹1,139.3 crore for the same period. TCI\'s consolidated total income was ₹1,150.6 crore, compared to ₹1,056.0 crore for the corresponding quarter of the previous year. Total expenses for the quarter amounted to ₹1052.5 crore, compared to ₹974.5 crore in the same quarter last year. Catch Shailendra Bhatnagar in the earnings conversation with Vineet Agarwal, Managing Director, TCI decoding the Q1 results", \'heading:Share Market News: Markets Falls On Sensex Expiry | Weak Q1 Results | FII Selling In Markets\', \'content:Nifty Is Expected To Remain Under Pressure As Bulls Stay On The Sidelines Amid Weak Q1 Earnings And Continuous FPI Selling. The Market Has Witnessed Four Consecutive Weeks Of Net Selling In July Following A Four-Month Rally. Traders Continue To Press Sales As Valuations Re-Adjust, While Long Trades Fail To Gain Traction. With Monthly Expiry Approaching, Some Short Covering Could Be Seen On Wednesday And Thursday, But The Overall Sentiment Remains Weak.\', \'heading:3 Key Reasons Why Nifty Realty Slipped 4% In Trade | What Should Be Your Investment Strategy?\', \'content:Nifty Realty index slipped nearly 4% as stocks like Macrotech Developers, Oberoi Realty, and DLF faced sharp selling pressure. Despite decent earnings growth on a year-on-year basis, the numbers failed to meet the high expectations set by strong pre-sales figures, denting investor sentiment. Experts believe this dip could be a buying opportunity as the real estate sector may regain momentum in the latter half of the year, supported by favorable repo rate impacts. Should you accumulate top realty stocks like DLF, Lodha, and Shobha Developers at lower levels? Market Expert Gaurav Sharma Decodes the moves and investment strategy \', \'heading:TCS 12,000 Job Cuts | Large-Cap vs Mid-Cap IT | Impact on Infosys, HCL, TechM & Market Outlook\', \'content:TCS is reportedly planning its biggest-ever layoff of 12,000 employees over the next 12 months, sparking concerns about AI-led job cuts in the IT sector. What does this mean for large-cap IT giants like Infosys, HCL, Tech Mahindra and mid-cap IT players such as Mphasis and Coforge? Will other companies follow TCS in job reductions, and is this just the beginning of a bigger trend? Market Expert Gaurav Sharma discuss the potential impact on IT stocks, market sentiment, and whether investors should focus on large-cap IT, mid-cap IT, or a balanced portfolio. Watch to know the best investment strategy for IT stocks after the latest developments. \', \'heading:HOT STOCKS\', \'heading:TOP VIDEOS\', \'new_source_stocknews1\', \'heading:COMPANIES\', \'heading:NEWS\', \'heading:Bajaj Housing Finance reports strong Q4: Net Profit at Rs 587 cr, NII up 24%\', \'content:Bajaj Housing Finance reported that its gross non-performing assets (GNPA) of 0.29% for the quarter ended March 2025, marginally higher than 0.27% in the December quarter.\', \'heading:SBI Mutual Fund adjusts portfolio with key stock changes in March; check what all it added\', \'content:SBI Mutual Fund, with an AUM of Rs 10.67 lakh crore, tweaked its portfolio in March by increasing stakes in 350 stocks, reducing them in 169, and adding 26 new funds. The fund remains a significant player in the mutual fund market.\', \'heading:ICICI Bank Q4 profit jumps 18% YoY to Rs 12,630 cr; FY25 PAT at Rs 47,227 cr\', \'content:ICICI Bank’s asset quality improved in the March 2025 quarter, with the gross non-performing asset (NPA) ratio declining to 1.67 percent, compared to 1.96 percent in the December 2024 quarter and 2.16 percent a year ago.\', \'heading:HDFC Bank Q4 net profit soars\\xa06.7% YoY to Rs 17,616 cr, beats estimates; board recommends Rs 22 dividend\', \'content:Asset quality showed mixed trends. The gross non-performing asset (NPA) ratio improved to 1.33 percent as of March 31, 2025, down from 1.42 percent at the end of December 2024, but higher than 1.24 percent a year ago.\', \'heading:YES Bank Q4 profit jumps 64% YoY to Rs 738 cr on lower provisions, improved asset quality\', \'content:On the asset quality front, YES Bank reported gross non-performing assets (NPAs) of Rs 3,935.6 crore, with the gross NPA ratio improving to 1.6% from 1.7% a year earlier.\', \'heading:HDFC Bank Q4 result today: PAT may rise by 7% YoY, NII may surge up to 9%\', "content:HDFC Bank\'s Q4 results are anticipated to show a 7% YoY growth in Profit After Tax (PAT) and a potential 9% rise in Net Interest Income (NII), according to estimates by leading brokerages. The private lender\'s asset quality and credit costs are also expected to improve.", "heading:Jio Financial\'s Q4 profit rises 2% YoY to Rs 316 crore, Rs 0.5 dividend announced", \'content:Jio Financial records a 2% annual profit increase to Rs 316 crore, with an 18% revenue growth, and declares a dividend of Rs 0.5 per share.\', \'heading:Vijay Shekhar Sharma forgoes 2.1 crore Paytm ESOPs amid SEBI notice\', \'content:Paytm founder and CEO Vijay Shekhar Sharma voluntarily relinquished 2.1 crore ESOPs granted to him under the One97 Employees Stock Option Scheme, 2019.\\r\\n\\r\\n\', \'heading:IndusInd Bank faces Rs 1,979 cr blow to net worth after derivatives review\', \'content:In a regulatory filing, the lender confirmed that the financial implications of this derivatives issue will be reflected in its FY25 financial statements.\', \'heading:Sebi bars Gensol Engineering, its promoters from accessing markets in fund diversion case\', \'content:SEBI has also prohibited both Anmol and Puneet Singh Jaggi from serving as directors or key managerial personnel at Gensol during the period of the restriction.\', "heading:Godrej Industries\' chemicals segment acquires Savannah Surfactants", "content:Godrej Industries\' Chemicals Business has acquired Savannah Surfactants\' Food Additives Business to expand its global offerings in the Food & Beverages Industry. Located in Goa, Savannah boasts a manufacturing capacity of 5,200 MTPA. The acquisition aligns with Godrej\'s strategy for sustainable growth.", \'heading:Vedanta Faces Rs 71 crore fine over fly ash disposal; company to contest order\', \'content:Vedanta Limited has been fined Rs 71.16 crore by Odisha State Pollution Control Board for alleged unauthorised fly ash disposal.\', \'heading:PwC submits IndusInd derivatives review draft, bank denies receiving it\', \'content:As per news reports, the review, commissioned after the bank uncovered discrepancies in the booking of forex derivative transactions, focuses strictly on accounting aspects. \', "heading:\'Potential multibagger gains from our PSU shares...\': Expert on DIPAM\'s latest plan for PSU stocks", \'content:DIPAM noted that in the fiscal year 2025, Central Public Sector Enterprises (CPSEs) reported a historic dividend payout of Rs 1.5 trillion, leading to the government receiving Rs 74,016 crore in dividends. \', \'heading:Banking and Finance stocks in 2025: Winners, losers, and the road ahead after RBI’s rate cut\', \'content:Data from ACE Equity shows Bajaj Finance is the top performer in finance and banking space with a robust 29% YTD return till April 8, 2025\', \'heading:Up to 41% decline in 5 Days: here are 10 biggest losers in BSE500\', \'content: Data from ACE Equity reveals that both the BSE Sensex and BSE500 fell by 6% during the first five trading sessions of April. Meanwhile, select companies from the BSE500 index witnessed stock price declines of up to 41%\', \'heading:IndusInd Bank reports mixed Q4 results amid derivative accounting lapse\', "content:IndusInd Bank\'s Q4 report shows a 5.2% drop in loan growth while deposit expansion remains flat, following an accounting issue in its derivatives portfolio", "heading:\'Indian auto sector looks promising...\': Expert on investing in domestic auto stocks", \'content:The Nifty Auto Index has experienced a significant decline of almost 28% since reaching its peak in September 2024. The reasons for this downturn are evident, including concerns about US tariffs, a decrease in automobile demand, and a wider market correction.\', "heading:Bajaj Finance: \'If you had invested Rs 1 cr in stock...\', expert highlights stock\'s recent performance \\xa0", \'content:On Wednesday, Bajaj Finance Ltd stock experienced a 2.3% increase, reaching a 52-week high of Rs 8,879 per share on the NSE.\\xa0\', "heading:\'If you had put Rs 10 lakh in Adani Enterprises in 2015...\': Expert explains long-term investing benefits", "content:Among the Adani Group\'s listed companies, Adani Enterprises, the flagship company, witnessed a 40% decline in its stock price, currently trading at Rs 2,252 from a high of Rs 3,743 in June 2024 (as of March 18, 2025).", \'heading:IndusInd\\xa0Bank: Promoters will infuse capital into bank if and when required, says Ashok Hinduja\', "content:Hinduja emphasized that this is an opportune moment for the promoters to bolster the bank\'s capital base, especially in light of the recent decrease in the bank\'s share value following the revelation of discrepancies in the derivative portfolio last week.", \'heading:HOT STOCKS\', \'heading:TOP VIDEOS\', \'new_source_stocknews2\', \'heading:COMPANIES\', \'heading:NEWS\', "heading:Bharti Airtel, ITC, DLF, RailTel, MCX among stocks in focus next week; here\'s why", \'content:ITC stock will be in the spotlight next week after the company, post market hours, announced a 2.9 per cent year-on-year (YoY) rise in its net profit for the first quarter of FY26.\', \'heading:Q1 results next week: Bharti Airtel, DLF, SBI, Tata Motors, LIC, Titan; check full list\', \'content:The week for quarterly earnings will kick off on Monday, August 4. Investors will track the results of DLF Ltd, a major real estate developer. \', \'heading:Coal India, IOC, GAIL, Mankind, Britannia, Berger Paints to turn ex-dividend next week\', \'content:The IOC board, at its meeting held on April 30, had recommended a final dividend of Rs 3 per share for FY25 of a face value of Rs 10 each. \', \'heading:Netweb Tech shares jump 17% in 4 days; is there more upside ahead?\', \'content:Netweb Tech: The stock surged 4.95 per cent to settle at Rs 2,133.70. At this closing price, it has gained 16.86 per cent in four trading days.\', \'heading:ITC Q1 earnings: Net profit at Rs 5,343 crore, revenue rises 20%\\xa0\', \'content:Net profit came at Rs 5343 crore in the June quarter against Rs 5092 in the corresponding period a year earlier.\\xa0\', \'heading:Paras Defence shares tumble 22% in a month amid continued sell-off; what lies ahead?\', \'content:Paras Defence: Both BSE and NSE have placed the stock under the long-term Additional Surveillance Measure (ASM) framework -- a step typically taken to alert investors amid increased price volatility.\', \'heading:Dead economy? Rahul Gandhi makes up to 67% return on select stocks as India stays strong\', "content:At least nine of Gandhi\'s two dozen portfolio stocks including Divi\'s Laboratories Ltd, Garware Technical Fibres Ltd, Dr Lal Pathlabs Ltd, ICICI Bank Ltd and Bajaj Finance Ltd delivered him up to 67 per cent returns.", \'heading:KIOCL shares surge 14%; analysts see more upside ahead\', \'content:KIOCL: The scrip witnessed heavy trading volume along with the price action today as around 20.28 lakh shares changed hands on BSE.\', \'heading:Tata Power Q1 earnings: Net profit rises to Rs 1,262 crore, EBITDA climbs 17% \', \'content:Tata Power earnings: Net profit rose to Rs 1,262 crore in the first quarter against Rs 1189 crore profit in the June 2024 quarter. \', "heading:Lupin, SBI, Bajaj Finance among Axis Securities\' top 15 stock picks with up to 30% upside potential", \'content:Topping the list is pharmaceutical major Lupin Ltd, with an expected upside of 30 per cent. It is followed by State Bank of India (SBI), offering 29 per cent potential gains and Bajaj Finance, with 25 per cent upside.\', \'heading:Q1 earnings: Graphite India shares plunge 8% as EBITDA, net profit fall \', "content:Graphite India\'s shares dropped 8% as Q1 results showed a 60% fall in EBITDA, with profit declining to ₹133 crore. ", \'heading:Choice AMC receives Sebi nod to launch mutual fund operations\', \'content:With the regulatory clearance in place, Choice is set to commence operations of its AMC, initiating a strategic and phased rollout beginning with passive investment products such as index funds and exchange traded funds (ETFs).\', "heading:HUL shares end 1% higher after 8% intraday surge; what\'s ahead?", \'content:HUL: On the earnings front, the FMCG major posted a standalone net profit of Rs 2,732 crore for the first quarter of FY26, marking a 7.6 per cent increase from Rs 2,538 crore in the same period last year.\', \'heading:Dixon Technologies: Low debt, high profit CAGR stock is overbought on charts; buy, sell or hold?\', \'content:Dixon Technologies share price: The stock has rallied 14.36% in a month, adding to the gains of 54% in nearly a year from its 52 week low. \', \'heading:Sensex tanks 585 pts, Nifty below 24,600; why stock market is losing steam\', \'content:At close, the BSE Sensex was down 585.67 points, or 0.72 per cent, at 80,599.91. The NSE Nifty50 declined 203 points, or 0.82 per cent, to end at 24,565.35.\', \'heading:Suzlon Energy shares jump 8% on 381 MW order win from Zelestra; what analysts are saying\', "content:Suzlon: Today\'s sharp rise came after the renewable energy solutions provider informed bourses that it secured a 381 MW order from Zelestra India and its affiliates.", "heading:ITC shares gain ahead of Q1 results; here\'s earnings preview   ", \'content:ITC Q1 results preview: HSBC said ITC’s Q1 trends remain relatively stable. The cigarettes business could see mid-single digit revenue growth, but Ebitda growth could be lower, it said.\', \'heading:Netweb Tech shares jump 13% as Q1 profit doubles; stock up 75% from 52-week low\', \'content:Operating income for the quarter rose more than double to Rs 301.21 crore from Rs 149.32 crore in the corresponding quarter last year, Netweb Technologies reported.\', "heading:GRSE inks MoU with German firm, will benefit defence forces; here\'s how\\xa0", \'content:GRSE stock, which closed at Rs 2613 on July 31 was trading 1.26% lower at Rs 2,580 in the current session on BSE. \\xa0\', \'heading:Adani Power board approves stock split in 1:5 ratio\', \'content:"The Board has considered and approved the sub-division/split of the existing 1 (One) equity share of the Company having face value of Rs 10 (Rupees Ten only) each fully paid- up, into 5 (Five) equity shares having face value of Rs 2 (Rupees Two only) each, fully paid- up, subject to approval of the shareholders of the company," Adani Power stated in a BSE filing.\', \'heading:Adani Power Q1 earnings: Net profit falls to Rs 3,385 crore, board announces stock split    \', \'content:Adani Power stock fell 0.46% to Rs 584.70 in the current session against the previous close of Rs 587.40 on BSE. Market cap of the firm stood at Rs 2.24 lakh crore.\\xa0\', \'heading:HOT STOCKS\', \'heading:TOP VIDEOS\', \'new_source_stocknews3\', \'heading: Option trading for range-bound market: Shubham Agarwal\', \'content:Range bound markets are also equally profitable, if traded effectively using Options, explains Shubham Agarwal.\', \'content:\\n\', \'heading: Stocks slump on latest tariffs, soft jobs data\', \'content:The Dow Jones Industrial Average fell 542.40 points, or 1.23%, to 43,588.58, the S&P 500 lost 101.38 points, or 1.60%, to 6,238.01 and the Nasdaq Composite lost 472.32 points, or 2.24%, to 20,650.13.\', \'content:\\n\', \'heading: Market reports longest weekly losing streak in two years; rupee weakens\', \'content:The Indian rupee shed 100 paise to close at 87.52 per dollar on August 1 against the July 25 closing of 86.52.\', \'content:\\n\', \'heading: Chartist Talk: Sudeep Shah shares 2 buy and 2 sell picks amid market caution; Nifty at risk below 24,400\', \'content:The 24,400–24,350 zone is expected to act as an immediate support for Nifty. A sustained break below this could further accelerate the downside. On the flip side, the 50-day EMA zone of 24,900–24,950 now stands as a crucial hurdle, and any meaningful recovery would need to clear this zone with conviction, said Sudeep Shah of SBI Securities.\', \'content:\\n\', \'heading: Bulk deals: SR Foundation buys half a percent stake in Belrise Industries\', \'content:Belrise Industries shares has seen buying interest for fourth consecutive week, rising nearly 37 percent during the period.\', \'content:\\n\', \'heading: Top gainers & losers: Stocks that moved the most on August 1\', \'content:\', \'content:\\n\', \'heading: Business in the Week Ahead (August 4-8, 2025)\', \'content:Be it trade or monetary policy, the US reset is making many go back to the drawing board. What should be your market toolkit for the coming week? What are the markers you just can’t avoid? There are lots, apart from a couple of key credit policy calls, and the earnings dashboard. Hop on\', \'content:\\n\', \'heading: Taking Stock: Nifty below 24,600 in fifth week of fall, big selloff in pharma\', "content:Tracking weak global cues, the indices had opened lower but recovered, however, a mid-session profit booking and extended selloff in the final hour dragged Nifty below 24,550, ending near day\'s low.", \'content:\\n\', \'heading: Symphony Approves FY25 Financials, Declares ₹8/Share Final Dividend\', \'content:Symphony Approves FY25 Financials, Declares ₹8/Share Final Dividend\', \'content:\\n\', \'heading: Markets slide for second session amid global weakness, tariff Concerns\', \'content:Asian markets were largely in the red, with indices in South Korea, Japan, China and Hong Kong trading lower. The negative trend followed weak closing on Wall Street, and US futures indicating a subdued start, further dampening investor confidence.\', \'content:\\n\', \'heading: Textile stocks extend losses after Trump tariffs; Gokaldas Exports, Pearl Global, KPR Mill down up to 6%\', \'content:Textile stocks: Bangladesh has secured a 20 percent tariff deal with US. Indian textile exports will now face higher tariffs on exports than their rivals from Bangladesh.\', \'content:\\n\', \'heading: Live: Nifty ends fifth week lower, slips below 24,600-mark | Closing Bell\', "content:Nifty Extends Losing Streak To Fifth Week. Nifty Slips Below 24,700; Sensex Down 400 Pts. Barring FMCG, All Sectors Down In The Red. FMCG Best Sectoral Performer This Week. Realty, Bank Worst Performer This Week. Pharma Sees Severe Cuts On Trump\'s Letters. Broader Mkts Underperform; India Vix Up 2%. Nifty 50 Gainers: Trent, Asian Paints, Nestle. Nifty 50 Laggards: Sun Pharma, Dr Reddy\'s, Cipla. Join Lovisha Darad and Yatin Mota as they break down the week’s market action and highlight stocks to watch.", \'content:\\n\', \'heading: ABB India share price falls 2.01%\', "content:With the stock price at Rs 5,399.50, ABB India\'s performance reflects the current market dynamics and investor reactions within the NIFTY NEXT 50 landscape.", \'content:\\n\', \'heading: Bharti Airtel share price falls 1.39%\', "content:With the stock\'s last traded price at Rs 1,887.70, Bharti Airtel has seen a decrease of 1.39% in today\'s session.", \'content:\\n\', \'heading: Retail Losses, Jane Street & Weekly Options: What Needs Fixing in F&O?\', \'content:With retail investors suffering massive losses and fresh allegations of market manipulation involving Jane Street, the need to reform India’s derivatives market has never been more urgent. The challenge lies in striking the right balance—ensuring the market stays vibrant and efficient, without turning into a playground for speculation and manipulation.\\nMoneycontrol’s panel discussion on reforming the F&O segment brought together four seasoned experts to explore actionable solutions—from curbing excessive retail participation to rethinking weekly options.\', \'content:\\n\', \'heading: Adani Power announces its first-ever stock split: What it means for shareholders?\', \'content:Adani Power announces its first-ever stock split in the ratio of 1:5.\', \'content:\\n\', \'heading: BSE-listed smallcap declares final dividend for FY25\', \'content:The company previously set the record date for dividend eligibility on July 04, 2025.\', \'content:\\n\', "heading: ITC share price gain 1.27% in friday\'s trading session", "content:With the stock\'s last traded price at Rs 417.20, ITC has seen a gain of 1.27% from the previous close.", \'content:\\n\', \'heading: Smallcap NBFC to consider funds raising via private placement\', \'content:Smallcap NBFC to consider funds raising via private placement\', \'content:\\n\', \'heading: Sensex down 350 pts, Nifty below 24,650 as US unleashes tariffs; pharma, IT, metal stocks tumble\', "content:Laggards on the Nifty included Dr Reddy\'s, Sun Pharma, Cipla, Tata Steel, and ONGC.", \'content:\\n\', "heading: Adani Power Q1 results: Net profit falls 13.5% to Rs 3,385 crore, firm\'s board OKs 1:5 stock split", \'content:Adani Power Q1 results: Revenue rises 6% to Rs 14,109 crore\', \'content:\\n\', \'heading: Buy Transport Corporation of India; target of Rs 1350: Sharekhan\', \'content:Sharekhan is bullish on Transport Corporation of India has recommended buy rating on the stock with a target price of Rs 1350 in its research report dated July 30, 2025.\', \'content:\\n\', \'heading: Buy Amber Enterprises India; target of Rs 9300: Sharekhan\', \'content:Sharekhan is bullish on Amber Enterprises India has recommended buy rating on the stock with a target price of Rs 9300 in its research report dated July 30, 2025.\', \'content:\\n\', \'heading: Buy Bharat Electronics; target of Rs 445: Sharekhan\', \'content:Sharekhan is bullish on Bharat Electronics has recommended buy rating on the stock with a target price of Rs 445 in its research report dated July 30, 2025.\', \'content:\\n\', \'heading: Sell Mahanagar Gas; target of Rs 1266: Geojit Financial Services\', \'content:Geojit Financial Services is bearish on Mahanagar Gas has recommended sell rating on the stock with a target price of Rs 1266 in its research report dated July 31, 2025.\', \'content:\\n\', \'heading:Watch\', \'heading:You got 30 Day’s Trial of \']', name='get_topic_news', id='cce3c808-7ad7-4a7e-9eb3-2da0d5f8787f', tool_call_id='call_lQtAX0I2KBlN1mlfQ2hXd2Ub'), ToolMessage(content='[\'new_source_bankingnews\', \'heading: Federal Bank’s corporate loan book to grow 8-10% in FY26 on tariffs uncertainty, cheaper funds at bond market: CEO\', \'content:According to the investor presentation of the bank, corporate loan book of the bank increased by 6 percent on-year to Rs 83,680 crore in Q1FY26, from Rs 79,208 crore in a year ago period, and Rs 83,680 crore in a quarter ago period.\', \'heading: Federal Bank Q1 results: Net profit falls 15% on-year to Rs 862 crore\', \'content:The bank registered operating profit of Rs 1556.29 crore for the quarter ended June 30, 2025.\', \'heading: Financial bids for IDBI Bank stake by December quarter, winning bid likely by end of fiscal: DIPAM Secretary\', \'content:The government has completed the process of due diligence, and data room protocols have been completed for all interested parties, the secretary said, adding that formal consultations have been completed with all qualified and interested parties.\', \'heading: ICICI Lombard scaled down motor, fire insurance coverage over pricing pressure: CFO Gopal Balachandran\', \'content:ICICI Lombard CFO Balachandran told Moneycontrol growing those segments may not make financial sense, citing stagnant third-party premium rates over the past 4-5 years.\', \'heading: PSU banks dominated bond issuances in FY25, more Tier-II debt raise likely this fiscal\', \'content:Private lenders have been totally absent from hybrid bond issuances in FY25. Despite favorable conditions for Tier-2 debt, these banks chose to rely on internal accruals and equity-driven capital planning.\', \'heading: Distributors opting for short-term health plans over long term post commission rule change: Niva Bupa CEO\', \'content:Krishnan Ramachandran, CEO, says the health insurer’s internal goal is to grow 5-10 percentage points faster than the overall health insurance industry\', \'heading: If I were Allianz, I’d be more worried about retaining Go Digit than the other way around: Chairman Kamesh Goyal\\xa0\', \'content:Addressing concerns over the Allianz-Jio reinsurance JV, Goyal said Allianz intends to continue its partnership with Go Digit, and the existing contract allows no mid-term changes, Goyal says\', \'heading: IDFC First shareholders grant Warburg affiliate board seat, reversing earlier decision\', \'content:While Currant Sea’s board nominee has not yet been disclosed, this marks a significant change in shareholder sentiment from May 2025, when a similar resolution was voted down.\', \'heading: RBI’s fraud detection tool MuleHunter AI expands reach with over 15 more banks nearing rollout\', \'content:The banks that have implemented the MuleHunter AI are Canara Bank, Punjab National Bank, Bank of India, Bank of Baroda, and AU Small Finance Bank, Suvendu Pati added\', \'heading: Indian rupee opens 2 paise higher at 87.58/USD on lingering worries over Trump tariff\', \'content:The local currency opened at 87.58 against the US dollar as compared to 87.60 against the greenback at previous close. According to the Bloomberg data, the domestic currency has depreciated 1.70 percent in July.\', \'heading: Fed’s hard line may force the RBI to keep purse wide open\', \'content:The Monetary Policy Committee (MPC) can’t afford to sit on the fence—it’s time for bold monetary stimulus to support growth.\', \'heading: Loan against gold jewellery grew 124% in June, fastest among personal loans, RBI data shows\', \'content:In absolute terms, the outstanding loan against gold jewellery stood at Rs 2.77 lakh crore as on June 27 . There has been a decline in the consumer durable segment\', "heading: DCB Bank\'s June quarter net profit rises 20% on year at Rs 157 crore", \'content:The Gross NPA as on June 30, 2025 was at 2.98 percent, while Net NPA was at 1.22 percent. The Provision Coverage Ratio (PCR) came in at 74.04 percent and PCR without considering gold loan NPAs was at 75 percent.\', \'heading: PSU banks hit harder by compression on yield on advances in Q1FY26\', \'content:According to data compiled by Moneycontrol, yield on advances for PSU banks dropped by 15–71 basis points (bps), significantly more than the 12–30 bps reduction observed for private banks.\', \'heading: Indian bond yields open 2 bps higher as Trump announces 25% tariffs\', "content:US Fed\'s Federal Open Market Committee decided to keep interest rates unchanged at 4.25 percent to 4.5 percent in the July 29-30 meeting", \'heading: Rupee slides further, opens 28 paise down at 87.71 after Trump slaps 25% tariff, Russia penalty\', \'content:The rupee has depreciated 0.83% in the last two weeks. This year, so far, the currency is the worst performing among Asian peers, down 3.25%\', \'heading: Indian rupee may open close to 88-mark against US dollar as Trump announces 25% tariffs on India\', \'content:According to the Bloomberg data, Indian rupee has depreciated 0.83 percent in last two weeks. On a year-to-date basis, the local currency is under pressure and second worst performing among Asian peers. It is down 3.25 percent, Bloomberg data showed.\', \'heading: Rupee at over at four-month low at 87.43 against dollar on Trump’s tariff threat\', \'content:The Rupee ended at 87.4250 against the US dollar compared to 86.8200 against the greenback at previous close, lower by 61 paise down on July 30.\', \'heading: Capital adequacy of PSU banks outpaces that of private lenders amid fundraising efforts\', \'content:Analysts expect capital buffers to remain strong across the banking sector. ICRA projects a 40–50 basis point increase in Tier-I capital ratios for PSBs this fiscal\', \'heading: PSU lenders’ average valuation surged 225% over five years on improved financials and asset quality\', \'content:According to data compiled from 11 major PSU banks, their average price-to-book (P/B) ratio has jumped from 0.35 in March 2020 to 1.14 in July 2025.\', \'heading: Merchant bankers tasked to bring foreign capital into PSU banks during upcoming QIPs\', \'content:The government budgets to earn approximately Rs 20,000 crore through QIP of five banks, namely Bank of Maharashtra, Indian Overseas Bank, UCO Bank, Central Bank of India, and Punjab and Sind Bank.\', "heading: Rupee falls to weakest since mid-March fearing impact of Trump\'s tariff on exports", \'content:The weakness is likely due to worries over the potential impact of the tariff rate on exports, though Reuters reported that there has likely been an intervention by the Reserve Bank of India that has helped limit losses.\', \'heading: Chart of the Day | India’s digital payments are surging, so are the risks\', \'content:As India’s digital payments soar to new heights, the journey to a cashless economy faces critical tests in security, inclusion, and regulation.\', \'heading: MPC Poll | RBI may hold rates in August policy to assess new data on inflation\', \'content:The central bank, so far has reduced the repo or the benchmark rate by 100 basis points (Bps) since February to aid growth, with 25 bps each in February and April respectively, and 50 bps in the June policy.\', \'heading: Not in a hurry\\xa0to burn our fingers, says CEO Aggarwal after Shriram General gives crop tender bids a miss\', \'content:Anil Kumar Aggarwal says the insurer is targeting a full-year premium of Rs 4,600 crore and could reach Rs 5,000 crore if demand picks up during the upcoming festival season\', \'heading: Rupee opens 18 paise down at 86.84 as dollar index surges\', "content:The dollar index, which measures the greenback\'s strength against a basket of six major currencies, rose to 98.64 in the early morning trade from below-97 levels in the previous session", \'heading: UPI’s free ride for all nearing its end?\', \'content:As RBI Governor Sanjay Malhotra signals the end of free UPI transactions, the move could reshape India’s fintech revolution, balancing innovation with sustainability\', \'heading: IndusInd Bank chairman assures investors about CEO hunt\', \'content:Chairman Mehta says, the bank has followed due process and timelines in submitting its recommendations, which remain consistent with its original proposal\', \'heading: IndusInd Bank sees further scope to cut savings, term deposit rates, says Soumitra Sen\', \'content:The cost of deposit of the bank in the reporting quarter eased by 9 basis points (Bps) year-on-year, and 6 bps on-quarter.\', \'heading: Why IRDAI wants insurers to appoint an in-house ombudsman and what it means for policyholders\', \'content:The insurance regulator has released a draft framework to mandate internal ombudsmen within insurance companies, and is expected to finalise the regulations by Q3FY26. Moneycontrol explains everything you need to know.\', \'heading: Double Trouble: IT layoffs & loan slowdown – is there a storm brewing?\', \'content:Private banks, which have higher exposure to private sector employees, particularly those from IT, have witnessed an 8-12 percent contraction in their FY26 EPS estimates.\', \'heading: MC Interview| Canara Bank to transfer 8 NPAs worth Rs 4,000 cr to NARCL by Q3: CEO K. Satyanarayan Raju\', \'content:Canara Bank’s total SMA 2 book was Rs 6,856 crore, of which Rs 5,000 crore pertain to two accounts—Rs 3,000 crore guaranteed by a state government for an irrigation project, and a rental discount real estate project in Bengaluru, Raju said.\', "heading: Aim to be among Top 3 players in India, says UBS\' Abhishek Joshi", "content:Betting on UBS and Credit Suisse\'s synergies,  Abhishek Joshi, Head of ECM India, UBS, is confident that the Indian market will remain attractive from an IPO and deals perspective this year", \'heading: Shriram Finance looks at new borrowing strategy to reduce costs\', \'content:Early onset of monsoon led to delay in construction activities to August and September, which had resulted in fall in AUM on construction equipment in Q1FY26, Revankar said.\', \'heading: Kotak Mahindra Bank Q1 FY26: a blip not a blow\', \'content:The bank had an unpleasant start to FY26\', \'heading: Rupee opens 5 paise up at 86.47 dollar as dollar index eases\', \'content:In the near term, the rupee is likely to trade in a the 86–87 range, experts say\', "heading: Banking Central | How to tackle rising frauds in India\'s banking sector", \'content:With frauds soaring to Rs 36,014 crore in 2024-25, the RBI’s latest data paints a grim picture of systemic cracks. Time for tougher oversight and accountability.\', \'heading: Kotak Mahindra Bank flags stress in vehicle loans, resumes microlending cautiously\', \'content:Bank sees MFI stress peaking, cautiously restarts disbursements across key States\', \'heading: Kotak Mahindra Bank Q1 Results: Net profit declines 7% to Rs 3,282 crore\', \'content:Provisions surge 109% yoy to Rs 1,208 crore; NII up 6% to Rs 7,259 crore\', \'heading: Zubin Mody resigns as CHRO of IndusInd Bank, cites pursuit of new opportunities\', \'content:In his resignation letter, Zubin Mody expressed that stepping down from the Bank was a difficult decision, marking the end of a fulfilling two-decade-long journey\', \'heading: DFS Secretary urge insurance brokers to help increase coverage in rural India, smaller cities\', \'content:Insurance brokers can augment insurance penetration through grassroots engagement, simplified product dissemination, and capacity building of local intermediaries, Nagaraju said.\', "heading: IRDAI\'s Satyajit Tripathy cautions on practices of insurance brokers to increase valuation", \'content:ripathy said that in the insurance broking space, there is a record level of activity in terms of mergers, acquisitions, and demergers in the distribution segment. “The regulator is witnessing increased activity in this area on a day-to-day basis,” he said.\', \'heading: Bank of Baroda Q1 net profit rises 1.9% on-year to Rs 4,541 crore\', "content:During the June quarter, BoB\'s total interest income rose 4.9 percent on-year to Rs 31,091 crore compared to Rs 29,629 crore a year ago.", \'heading: Vault Matters | Why now isn’t the time to upgrade banks\', \'content:While there are at least four bank requests pending for regulator’s blessings to get a bump up in the value chain, now is the time to observe and assess, not react and regret\\xa0\', \'heading: RBI to consolidate 8,000 regulations, set up review cell, says governor Sanjay Malhotra\', \'content:The move is aimed as simplifying compliance and enhancing clarity. The review cell will revisit all regulations every five–seven years to weed out obsolete norms and examine if there’s the right balance between efficiency and financial stability\', \'heading: Indian rupee opens 16 paise lower at 86.57/USD on dip in Asian currencies, eye on FOMC next week\', \'content:Asian currencies such as South Korean Won was down 0.27 percent from previous close, Philippine Peso was down 0.26 percent, Japanese Yen and Indonesia Rupiah was down 0.12 percent each, and Thai Baht was down 0.05 percent, as per Bloomberg data.\', \'heading: Bajaj Finance’s 2 & 3-wheeler, MSME businesses to grow slow in FY26: MD Rajeev Jain\', \'content:Two & Three-Wheeler Finance AUM declined by 20 percent on-year to Rs 15,703 crore in April-June quarter, from Rs 19,689 crore in a year ago period.\', \'heading: Bajaj Finance to submit detailed succession planning in six months to board: MD Rajeev Jain\', \'content:While deliberations on succession plan would start in 6 - 8 months, Jain to remain at the helm of affairs till March 2028.\', \'heading: Bajaj Finance curtails Karnataka exposure by up to 50%, amid political, regulatory risk: MD Rajeev Jain\', \'content:Karnataka accounted for 11 percent of the company’s balance sheet, making this a significant recalibration\', \'heading: Bajaj Finance reports Rs 1,556 crore microfinance loans in Q1FY26 after separating MFI from Rural B2C Portfolio\', \'content:In April-June quarter, Bajaj Finance has separated microfinance business from the Rural B2C loans. The company is sourcing MFI business through MFI branches.\', \'heading:Watch\', \'heading:You got 30 Day’s Trial of \', \'new_source_todaymarket\', \'heading:COMPANIES\', \'heading:NEWS\', \'heading:DCB Bank Q1FY26 | Net Profit Up 20% At Rs 157 Cr | Top Management Excl\', "content:DCB Bank\'s standalone net profit jumped 19.71% to Rs 157.26 crore in Q1 FY26, compared with Rs 131.36 crore in Q1 FY25. Total income increased 25.57% to Rs 2,049.69 crore in Q1 FY26 as against Rs 1,632.25 crore in Q1 FY25. Net interest income (NII) climbed 16.90% to Rs 581 crore during the quarter, compared with Rs 497 crore posted in corresponding quarter last year. Deposits stood at Rs 62,039 crore as of 30 June 2025, registering the growth of 20.02%, compared with Rs 51,690 crore as of 30 June 2024. Net advances grew 21.41% year on year to Rs 51,215 crore in Q1 FY26. As of 30 June 2025, CASA ratio was at 23.32% as against 25.41% as of 30 June 2024. Gross non-performing assets (NPA) stood at Rs 1,553.63 crore as on 30 June 2025 as compared with Rs 1,353.47 crore as on 30 June 2024. Catch Shailendra Bhatnagar with Q1 earnings conversation with Praveen Kutty, MD & CEO, DCB Bank decoding the bank results", "heading:Tuhin Kanta Pandey: SEBI To Enforce \'@valid\' UPI For Safer Market Transactions From Oct 1", \'content:SEBI Chairman Tuhin Kanta Pandey, speaking at an ICAI event, highlighted a key investor protection measure being operationalised by SEBI. From October 1, 2025, financial transactions with SEBI-registered intermediaries via UPI must be routed through a verified domain—“@valid”—developed in coordination with NPCI. While the initiative has been under development, it is now being formally rolled out to raise awareness and enhance transaction security in the securities market. The “@valid” UPI handle will help investors verify the authenticity of intermediaries and reduce the risk of fraud, ensuring secure, regulated payment channels across all UPI-linked investment platforms. \\r\\n\', \'heading:US President Donald Trump Taunts India, Backs Pakistan Pact Amid Tariff Row\', \'content:Donald Trump’s latest comments have stirred controversy as he hints at a trade pact with Pakistan while criticizing India’s ties with Russia. Trump slammed India’s “high tariffs” and dismissed prospects of stronger trade relations, claiming the US has “done very little business” with India. Mr. Sabnavis reacts, calling it a ploy under Trump’s “Make America Great Again” agenda – aimed at rattling trade partners and gaining leverage. Despite these remarks, Sabnavis asserts India’s strong sovereign stance and ability to protect its economic interests. As the geopolitical drama unfolds, questions rise on how India will navigate trade talks with the US while safeguarding strategic autonomy and export competitiveness amid a volatile global order. \', \'heading:Will Tata Motors Slip Below ₹600? Deven Choksey On $4B Iveco Deal\', \'content:Tata Motors’ $4 billion IVECO deal is set to reshape its global ambitions. Deven Choksey explains how this transaction could expand Tata Motors’ commercial vehicle business to over ₹2 lakh crore in size, tripling its scale. The deal brings advanced technology and a stronger global footprint, boosting long-term growth prospects. While short-term market adjustments may occur, Choksey believes this is a big opportunity for Tata Motors as its passenger and commercial vehicle divisions undergo demerger. Will the stock hold above ₹600, and what does it mean for EPS? Watch this detailed breakdown of the deal’s impact on Tata Motors’ global strategy, profitability, and investor outlook. \', "heading:Deven Choksey Explains Jio\'s ₹15,000 Cr Fund-Raise Move and Insurance Gameplan", \'content:Reliance Jio has announced the allocation of 50 crore warrants at ₹316.50 to the promoter family, raising nearly ₹15,000 crore. But why is Jio raising such a big amount? Market expert Devan Choksey breaks down the reason behind this move and what it means for Jio’s future plans. The funds are likely to be deployed for Jio’s ambitious ventures in financial services, including its joint reinsurance business with Allianz. Reinsurance is a highly capital-intensive sector, and this investment could make Jio-Allianz one of India’s largest players in the space. Watch as Devan Choksey explains the strategy, growth potential, and long-term impact of this stake hike on Reliance Jio’s expansion plans. \', \'heading:KPIT Tech Q1 Result: Strong Deal Wins Of $241 Mn | Top Management Decodes The Growth, Outlook & More\', \'content:KPIT technologies Q1 Results: Reported its Q1 FY26 results with revenue at ₹1,538.8 crore, up 12.8% YoY, while net profit fell to ₹171.9 crore. The company retained its FY26 guidance of 18–22% revenue growth in constant currency and an EBITDA margin target of ~20.5%. Q1 saw strong deal wins worth US\\u202f$241 million TCV, largely in powertrain and connected vehicle projects across the US and Europe. Despite delays in execution timelines, the overall deal pipeline expanded ~20% YoY, reflecting strong client traction. The stock gained around 4% on the positive outlook and stable margins. Catch Shailendra Bhatnagar in conversation with Sachin Tikekar, JT MD & Co-founder, KPIT Technologies and Priya Hardikar, CFO, KPIT Technologies decoding Q1 results, outlook and more\', \'heading:Trump’s 25% Tariffs Could Hurt India’s Export Edge\', \'content:In this exclusive interview with Business Today TV, Ranen Banerjee, Partner and Economic Advisory Leader at PwC, discusses India’s tariff landscape and macroeconomic outlook. He notes that tariff uncertainty is reducing, but steep duties of 20–25% could still challenge India’s export competitiveness. While the pharma sector remains tightly coupled, other industries have room to diversify trade with the US. He speaks on the RBI’s neutral stance, subdued private investment, food inflation trends, and the rising cost of oil imports due to the Russia situation. Banerjee also underlines the need for continued government CAPEX to address high logistics costs and fuel long-term growth. \\r\\n\', \'heading:Tata Motors Slips 4% On Reports Of $4.5 Billion Iveco Deal\', "content:Tata Motors is reportedly considering a $4.5 billion bid to acquire Italian truck maker Iveco, in what could be the company’s biggest overseas deal since Jaguar Land Rover. We discussed the implications of this potential acquisition with Sharad Avasthi, Head of Research (PCG) at SMIFS. He shared insights on whether this could be Tata Motors\' next JLR moment, the possible benefits of global CV technology, and the strategic value of entering the premium truck segment. With Iveco already doubling in value this year and Tata Motors stock under pressure, we decode what this move could mean for shareholders, global expansion, and the future of the commercial vehicle business. ", \'heading:BOI Q1 Results: Profit Surges 32%, Stock Up 3% | CEO Rajneesh Karnatak On Strong Q1 & Outlook\', \'content:In this special episode of Easynomics on Business Today TV, Group Editor Siddharth Zarabi speaks with Professor Bharat Bhasker, Director of IIM Ahmedabad, about a milestone in Indian management education—the launch of IIM Ahmedabad’s first international campus in Dubai this September. Professor Bhasker shares why Dubai was chosen, the strategic focus on executive education, and how the new campus positions IIMA as a bridge to the Global South. The conversation delves into case-based pedagogy, AI-driven immersive learning, growing classroom diversity, and the institute’s response to a rapidly changing global business landscape. From fee structures and student intake to future expansion plans, this in-depth discussion offers rare insight into how India’s premier B-school is going global while staying true to its core mission of impact-led leadership. \', \'heading:Asian Paints Q1 Results: Top Management Decodes Expansion Plans, Growth Chart, Competition & More\', \'content:Asian Paints on Tuesday reported a 6% year-on-year (YoY) decline in its consolidated net profit for the June quarter at Rs 1,100 crore, compared to Rs 1,170 crore in the year-ago period. Revenue from operations marginally decreased by 0.20% YoY to Rs 8,924 crore. However, the company experienced a significant 59% sequential increase in PAT and a 7% QoQ rise in revenue compared to the previous quarter. Home Decor categories declined during the quarter with pressure on household disposable incomes. The International business saw value growth of 8.4% during the June quarter. Catch Earnings conversation with Shailendra Bhatnagar in conversation with Amit Syngle, MD & CEO, Asian Paints decoding the Q1 results, expansion plans and International Business growth \', "heading:InCred\'s Top Healthcare & Pharma Picks For The Next Decade", \'content:In this episode of Market Guru, Aditya Khemka, Fund Manager at InCred AMC, shares his top three high-conviction bets in the healthcare and pharma space for the next decade. He explains why Healthcare Global (HCG) is his preferred pick in the hospital segment, citing its strong focus on oncology and long-term patient retention. In the CDMO/API space, he highlights Jubilant Pharma for its niche offerings, high entry barriers, and solid execution track record. Lastly, he names FDC Limited as a standout in branded generics, thanks to its well-established household brands and potential for exponential growth. Watch as Khemka breaks down each company’s strengths and long-term value proposition for investors. \', \'heading:Asian Paints Q1 Results: Why Analyst Are Cautious On Asian Paints Despite Strong Q1 Sales Volumes\', \'content:Asian Paints has reported better-than-expected Q1 numbers with a 3.9% volume growth, surpassing market estimates of around 2%. The management commentary was also upbeat, hinting at stabilizing market share and improving industry momentum. However, despite these positives, Sudip Bandyopadhyay, Group Chairman, Inditrade Capital analyst remains cautious on taking fresh positions in the stock, citing rising competition from large conglomerates like JSW and Birla Opus and ongoing industry consolidation. Listen in \', \'heading:Motherson Sumi Wiring Q1 Results | CFO & COO On Profit Dip, EV Growth & Expansion Plans\', \'content:Business Today speaks to Gulshan Pahuja, Chief Financial Officer, and Anurag Gahlot, Chief Operating Officer of Motherson Sumi Wiring India Ltd (MSWIL), to discuss the company’s Q1 FY26 performance. Despite a 4% year-on-year decline in net profit to ₹143 crore, the company reported strong revenue growth of 14.2% to ₹2,494 crore, driven by new vehicle model launches and increased content per vehicle. The management addresses the reasons behind the dip in operating margins, which narrowed to 9.8% due to ramp-up costs at Greenfield projects. They also share insights into MSWIL’s growing electric vehicle (EV) segment, which now contributes 5.4% of total revenue, and reiterate their commitment to future-ready capacity expansion in alignment with OEM demand. The discussion also highlights the company’s continued debt-free status and its strategic focus areas for the upcoming quarters. \', \'heading:Aditya Khemka Of InCred AMC Creates Wealth For Investors With Pharma Small Caps & Patient Stock Pick\', \'content:On this episode of Market Guru by Business Today, we are joined by Aditya Khemka, Fund Manager at InCred Asset Management, who offers a rare and incisive look into his high-performing ₹550 crore healthcare portfolio. Known for his deep expertise in the pharma and diagnostics space, Mr Khemka explains what is fuelling the ongoing bull run in pharma stocks, and why hospital chains and branded generics form the core of his concentrated, small-cap-heavy strategy. In a wide-ranging conversation, he addresses concerns about pricing power in the hospital sector, the impact of potential US tariffs on Indian pharma exports, and why companies like Healthcare Global, Jubilant Pharma, and FDC are his top long-term bets. Mr Khemka also shares valuable insights on market valuations, sectoral trends, and his approach to bottom-up investing. An unmissable session for serious investors looking to understand the future of Indian healthcare investing.\', \'heading:Waaree Energies Q1 Results: Profit Zooms 92% YoY | Top Mgmt Decodes Q1 Highlights & Growth Plan\', \'content:Waaree Energies Q1 Results: The company posted a quarterly revenue of ₹4,597.18 crore, marking a 31.48% YoY growth, with EBITDA at ₹1,168.67 crore (up 82.61% YoY) and margins at 25.42%. Profit After Tax stood at ₹772.89 crore, a sharp 92.68% YoY rise. Key developments include a proposal to change the IPO object clause for shifting manufacturing from Odisha to Gujarat and Maharashtra, commissioning plans for additional module manufacturing capacities of 1.6 GW in Texas (USA) and 3.2 GW in Chikhli (Gujarat), and ongoing construction of green hydrogen, inverter, and battery energy storage facilities. Waaree Solar Americas Inc. recorded an order inflow of ~2.23 GW. The board also approved an additional capex of ₹2,754 crore for expanding cell capacity by 4 GW in Gujarat and ingot-wafer capacity by 4 GW in Maharashtra. Catch Sakshi Batra in conversation with Dr. Amit Paithankar, CEO & Whole-time Director, Waaree Energies Limited decoding the Q1 results and growth plan .\', \'heading:Daily Calls LIVE: Ask Your STOCK MARKET TODAY QUERIES | Market Update LIVE | Share Market News Today\', "content:BTTV brings you a new market show - \'Daily Calls,\' where you can gain invaluable insights and clarity on your market queries through our live sessions featuring expert analysts. Whether you\'re confused about where to invest, how to invest, or how to build and structure your portfolio.", \'heading:Maharashtra Temples & Trusts Can Now Invest In Stock Markets, Mutual Funds! Check Details\', "content:Maharashtra\'s iconic temples and charitable trusts could soon become active investors in India’s financial markets. With a sweeping reform effective July 21, 2025, Maharashtra state government has opened the gates for public trusts to allocate up to 50% of their corpus in mutual funds and market-linked securities. For decades, public trusts—be it religious institutions, social welfare organizations, or educational charities—were largely confined to traditional instruments such as fixed deposits and post office schemes. Any deviation into riskier assets required cumbersome, case-by-case permissions from the Charity Commissioner but that changes now. Catch Sakshi Batra explaining this mega move.", \'heading:TCI Q1 Results: Net Profit Rise 17% | Top Management Decodes The Growth Print For FY26\', "content:TCI Q1 Results: Transport Corporation of India Ltd (TCI) reported a consolidated net profit of ₹107.2 crore for the quarter ended June 30, 2025. The company\'s revenue from operations stood at ₹1,139.3 crore for the same period. TCI\'s consolidated total income was ₹1,150.6 crore, compared to ₹1,056.0 crore for the corresponding quarter of the previous year. Total expenses for the quarter amounted to ₹1052.5 crore, compared to ₹974.5 crore in the same quarter last year. Catch Shailendra Bhatnagar in the earnings conversation with Vineet Agarwal, Managing Director, TCI decoding the Q1 results", \'heading:Share Market News: Markets Falls On Sensex Expiry | Weak Q1 Results | FII Selling In Markets\', \'content:Nifty Is Expected To Remain Under Pressure As Bulls Stay On The Sidelines Amid Weak Q1 Earnings And Continuous FPI Selling. The Market Has Witnessed Four Consecutive Weeks Of Net Selling In July Following A Four-Month Rally. Traders Continue To Press Sales As Valuations Re-Adjust, While Long Trades Fail To Gain Traction. With Monthly Expiry Approaching, Some Short Covering Could Be Seen On Wednesday And Thursday, But The Overall Sentiment Remains Weak.\', \'heading:3 Key Reasons Why Nifty Realty Slipped 4% In Trade | What Should Be Your Investment Strategy?\', \'content:Nifty Realty index slipped nearly 4% as stocks like Macrotech Developers, Oberoi Realty, and DLF faced sharp selling pressure. Despite decent earnings growth on a year-on-year basis, the numbers failed to meet the high expectations set by strong pre-sales figures, denting investor sentiment. Experts believe this dip could be a buying opportunity as the real estate sector may regain momentum in the latter half of the year, supported by favorable repo rate impacts. Should you accumulate top realty stocks like DLF, Lodha, and Shobha Developers at lower levels? Market Expert Gaurav Sharma Decodes the moves and investment strategy \', \'heading:TCS 12,000 Job Cuts | Large-Cap vs Mid-Cap IT | Impact on Infosys, HCL, TechM & Market Outlook\', \'content:TCS is reportedly planning its biggest-ever layoff of 12,000 employees over the next 12 months, sparking concerns about AI-led job cuts in the IT sector. What does this mean for large-cap IT giants like Infosys, HCL, Tech Mahindra and mid-cap IT players such as Mphasis and Coforge? Will other companies follow TCS in job reductions, and is this just the beginning of a bigger trend? Market Expert Gaurav Sharma discuss the potential impact on IT stocks, market sentiment, and whether investors should focus on large-cap IT, mid-cap IT, or a balanced portfolio. Watch to know the best investment strategy for IT stocks after the latest developments. \', \'heading:HOT STOCKS\', \'heading:TOP VIDEOS\', \'new_source_stocknews1\', \'heading:COMPANIES\', \'heading:NEWS\', \'heading:Bajaj Housing Finance reports strong Q4: Net Profit at Rs 587 cr, NII up 24%\', \'content:Bajaj Housing Finance reported that its gross non-performing assets (GNPA) of 0.29% for the quarter ended March 2025, marginally higher than 0.27% in the December quarter.\', \'heading:SBI Mutual Fund adjusts portfolio with key stock changes in March; check what all it added\', \'content:SBI Mutual Fund, with an AUM of Rs 10.67 lakh crore, tweaked its portfolio in March by increasing stakes in 350 stocks, reducing them in 169, and adding 26 new funds. The fund remains a significant player in the mutual fund market.\', \'heading:ICICI Bank Q4 profit jumps 18% YoY to Rs 12,630 cr; FY25 PAT at Rs 47,227 cr\', \'content:ICICI Bank’s asset quality improved in the March 2025 quarter, with the gross non-performing asset (NPA) ratio declining to 1.67 percent, compared to 1.96 percent in the December 2024 quarter and 2.16 percent a year ago.\', \'heading:HDFC Bank Q4 net profit soars\\xa06.7% YoY to Rs 17,616 cr, beats estimates; board recommends Rs 22 dividend\', \'content:Asset quality showed mixed trends. The gross non-performing asset (NPA) ratio improved to 1.33 percent as of March 31, 2025, down from 1.42 percent at the end of December 2024, but higher than 1.24 percent a year ago.\', \'heading:YES Bank Q4 profit jumps 64% YoY to Rs 738 cr on lower provisions, improved asset quality\', \'content:On the asset quality front, YES Bank reported gross non-performing assets (NPAs) of Rs 3,935.6 crore, with the gross NPA ratio improving to 1.6% from 1.7% a year earlier.\', \'heading:HDFC Bank Q4 result today: PAT may rise by 7% YoY, NII may surge up to 9%\', "content:HDFC Bank\'s Q4 results are anticipated to show a 7% YoY growth in Profit After Tax (PAT) and a potential 9% rise in Net Interest Income (NII), according to estimates by leading brokerages. The private lender\'s asset quality and credit costs are also expected to improve.", "heading:Jio Financial\'s Q4 profit rises 2% YoY to Rs 316 crore, Rs 0.5 dividend announced", \'content:Jio Financial records a 2% annual profit increase to Rs 316 crore, with an 18% revenue growth, and declares a dividend of Rs 0.5 per share.\', \'heading:Vijay Shekhar Sharma forgoes 2.1 crore Paytm ESOPs amid SEBI notice\', \'content:Paytm founder and CEO Vijay Shekhar Sharma voluntarily relinquished 2.1 crore ESOPs granted to him under the One97 Employees Stock Option Scheme, 2019.\\r\\n\\r\\n\', \'heading:IndusInd Bank faces Rs 1,979 cr blow to net worth after derivatives review\', \'content:In a regulatory filing, the lender confirmed that the financial implications of this derivatives issue will be reflected in its FY25 financial statements.\', \'heading:Sebi bars Gensol Engineering, its promoters from accessing markets in fund diversion case\', \'content:SEBI has also prohibited both Anmol and Puneet Singh Jaggi from serving as directors or key managerial personnel at Gensol during the period of the restriction.\', "heading:Godrej Industries\' chemicals segment acquires Savannah Surfactants", "content:Godrej Industries\' Chemicals Business has acquired Savannah Surfactants\' Food Additives Business to expand its global offerings in the Food & Beverages Industry. Located in Goa, Savannah boasts a manufacturing capacity of 5,200 MTPA. The acquisition aligns with Godrej\'s strategy for sustainable growth.", \'heading:Vedanta Faces Rs 71 crore fine over fly ash disposal; company to contest order\', \'content:Vedanta Limited has been fined Rs 71.16 crore by Odisha State Pollution Control Board for alleged unauthorised fly ash disposal.\', \'heading:PwC submits IndusInd derivatives review draft, bank denies receiving it\', \'content:As per news reports, the review, commissioned after the bank uncovered discrepancies in the booking of forex derivative transactions, focuses strictly on accounting aspects. \', "heading:\'Potential multibagger gains from our PSU shares...\': Expert on DIPAM\'s latest plan for PSU stocks", \'content:DIPAM noted that in the fiscal year 2025, Central Public Sector Enterprises (CPSEs) reported a historic dividend payout of Rs 1.5 trillion, leading to the government receiving Rs 74,016 crore in dividends. \', \'heading:Banking and Finance stocks in 2025: Winners, losers, and the road ahead after RBI’s rate cut\', \'content:Data from ACE Equity shows Bajaj Finance is the top performer in finance and banking space with a robust 29% YTD return till April 8, 2025\', \'heading:Up to 41% decline in 5 Days: here are 10 biggest losers in BSE500\', \'content: Data from ACE Equity reveals that both the BSE Sensex and BSE500 fell by 6% during the first five trading sessions of April. Meanwhile, select companies from the BSE500 index witnessed stock price declines of up to 41%\', \'heading:IndusInd Bank reports mixed Q4 results amid derivative accounting lapse\', "content:IndusInd Bank\'s Q4 report shows a 5.2% drop in loan growth while deposit expansion remains flat, following an accounting issue in its derivatives portfolio", "heading:\'Indian auto sector looks promising...\': Expert on investing in domestic auto stocks", \'content:The Nifty Auto Index has experienced a significant decline of almost 28% since reaching its peak in September 2024. The reasons for this downturn are evident, including concerns about US tariffs, a decrease in automobile demand, and a wider market correction.\', "heading:Bajaj Finance: \'If you had invested Rs 1 cr in stock...\', expert highlights stock\'s recent performance \\xa0", \'content:On Wednesday, Bajaj Finance Ltd stock experienced a 2.3% increase, reaching a 52-week high of Rs 8,879 per share on the NSE.\\xa0\', "heading:\'If you had put Rs 10 lakh in Adani Enterprises in 2015...\': Expert explains long-term investing benefits", "content:Among the Adani Group\'s listed companies, Adani Enterprises, the flagship company, witnessed a 40% decline in its stock price, currently trading at Rs 2,252 from a high of Rs 3,743 in June 2024 (as of March 18, 2025).", \'heading:IndusInd\\xa0Bank: Promoters will infuse capital into bank if and when required, says Ashok Hinduja\', "content:Hinduja emphasized that this is an opportune moment for the promoters to bolster the bank\'s capital base, especially in light of the recent decrease in the bank\'s share value following the revelation of discrepancies in the derivative portfolio last week.", \'heading:HOT STOCKS\', \'heading:TOP VIDEOS\', \'new_source_stocknews2\', \'heading:COMPANIES\', \'heading:NEWS\', "heading:Bharti Airtel, ITC, DLF, RailTel, MCX among stocks in focus next week; here\'s why", \'content:ITC stock will be in the spotlight next week after the company, post market hours, announced a 2.9 per cent year-on-year (YoY) rise in its net profit for the first quarter of FY26.\', \'heading:Q1 results next week: Bharti Airtel, DLF, SBI, Tata Motors, LIC, Titan; check full list\', \'content:The week for quarterly earnings will kick off on Monday, August 4. Investors will track the results of DLF Ltd, a major real estate developer. \', \'heading:Coal India, IOC, GAIL, Mankind, Britannia, Berger Paints to turn ex-dividend next week\', \'content:The IOC board, at its meeting held on April 30, had recommended a final dividend of Rs 3 per share for FY25 of a face value of Rs 10 each. \', \'heading:Netweb Tech shares jump 17% in 4 days; is there more upside ahead?\', \'content:Netweb Tech: The stock surged 4.95 per cent to settle at Rs 2,133.70. At this closing price, it has gained 16.86 per cent in four trading days.\', \'heading:ITC Q1 earnings: Net profit at Rs 5,343 crore, revenue rises 20%\\xa0\', \'content:Net profit came at Rs 5343 crore in the June quarter against Rs 5092 in the corresponding period a year earlier.\\xa0\', \'heading:Paras Defence shares tumble 22% in a month amid continued sell-off; what lies ahead?\', \'content:Paras Defence: Both BSE and NSE have placed the stock under the long-term Additional Surveillance Measure (ASM) framework -- a step typically taken to alert investors amid increased price volatility.\', \'heading:Dead economy? Rahul Gandhi makes up to 67% return on select stocks as India stays strong\', "content:At least nine of Gandhi\'s two dozen portfolio stocks including Divi\'s Laboratories Ltd, Garware Technical Fibres Ltd, Dr Lal Pathlabs Ltd, ICICI Bank Ltd and Bajaj Finance Ltd delivered him up to 67 per cent returns.", \'heading:KIOCL shares surge 14%; analysts see more upside ahead\', \'content:KIOCL: The scrip witnessed heavy trading volume along with the price action today as around 20.28 lakh shares changed hands on BSE.\', \'heading:Tata Power Q1 earnings: Net profit rises to Rs 1,262 crore, EBITDA climbs 17% \', \'content:Tata Power earnings: Net profit rose to Rs 1,262 crore in the first quarter against Rs 1189 crore profit in the June 2024 quarter. \', "heading:Lupin, SBI, Bajaj Finance among Axis Securities\' top 15 stock picks with up to 30% upside potential", \'content:Topping the list is pharmaceutical major Lupin Ltd, with an expected upside of 30 per cent. It is followed by State Bank of India (SBI), offering 29 per cent potential gains and Bajaj Finance, with 25 per cent upside.\', \'heading:Q1 earnings: Graphite India shares plunge 8% as EBITDA, net profit fall \', "content:Graphite India\'s shares dropped 8% as Q1 results showed a 60% fall in EBITDA, with profit declining to ₹133 crore. ", \'heading:Choice AMC receives Sebi nod to launch mutual fund operations\', \'content:With the regulatory clearance in place, Choice is set to commence operations of its AMC, initiating a strategic and phased rollout beginning with passive investment products such as index funds and exchange traded funds (ETFs).\', "heading:HUL shares end 1% higher after 8% intraday surge; what\'s ahead?", \'content:HUL: On the earnings front, the FMCG major posted a standalone net profit of Rs 2,732 crore for the first quarter of FY26, marking a 7.6 per cent increase from Rs 2,538 crore in the same period last year.\', \'heading:Dixon Technologies: Low debt, high profit CAGR stock is overbought on charts; buy, sell or hold?\', \'content:Dixon Technologies share price: The stock has rallied 14.36% in a month, adding to the gains of 54% in nearly a year from its 52 week low. \', \'heading:Sensex tanks 585 pts, Nifty below 24,600; why stock market is losing steam\', \'content:At close, the BSE Sensex was down 585.67 points, or 0.72 per cent, at 80,599.91. The NSE Nifty50 declined 203 points, or 0.82 per cent, to end at 24,565.35.\', \'heading:Suzlon Energy shares jump 8% on 381 MW order win from Zelestra; what analysts are saying\', "content:Suzlon: Today\'s sharp rise came after the renewable energy solutions provider informed bourses that it secured a 381 MW order from Zelestra India and its affiliates.", "heading:ITC shares gain ahead of Q1 results; here\'s earnings preview   ", \'content:ITC Q1 results preview: HSBC said ITC’s Q1 trends remain relatively stable. The cigarettes business could see mid-single digit revenue growth, but Ebitda growth could be lower, it said.\', \'heading:Netweb Tech shares jump 13% as Q1 profit doubles; stock up 75% from 52-week low\', \'content:Operating income for the quarter rose more than double to Rs 301.21 crore from Rs 149.32 crore in the corresponding quarter last year, Netweb Technologies reported.\', "heading:GRSE inks MoU with German firm, will benefit defence forces; here\'s how\\xa0", \'content:GRSE stock, which closed at Rs 2613 on July 31 was trading 1.26% lower at Rs 2,580 in the current session on BSE. \\xa0\', \'heading:Adani Power board approves stock split in 1:5 ratio\', \'content:"The Board has considered and approved the sub-division/split of the existing 1 (One) equity share of the Company having face value of Rs 10 (Rupees Ten only) each fully paid- up, into 5 (Five) equity shares having face value of Rs 2 (Rupees Two only) each, fully paid- up, subject to approval of the shareholders of the company," Adani Power stated in a BSE filing.\', \'heading:Adani Power Q1 earnings: Net profit falls to Rs 3,385 crore, board announces stock split    \', \'content:Adani Power stock fell 0.46% to Rs 584.70 in the current session against the previous close of Rs 587.40 on BSE. Market cap of the firm stood at Rs 2.24 lakh crore.\\xa0\', \'heading:HOT STOCKS\', \'heading:TOP VIDEOS\', \'new_source_stocknews3\', \'heading: Option trading for range-bound market: Shubham Agarwal\', \'content:Range bound markets are also equally profitable, if traded effectively using Options, explains Shubham Agarwal.\', \'content:\\n\', \'heading: Stocks slump on latest tariffs, soft jobs data\', \'content:The Dow Jones Industrial Average fell 542.40 points, or 1.23%, to 43,588.58, the S&P 500 lost 101.38 points, or 1.60%, to 6,238.01 and the Nasdaq Composite lost 472.32 points, or 2.24%, to 20,650.13.\', \'content:\\n\', \'heading: Market reports longest weekly losing streak in two years; rupee weakens\', \'content:The Indian rupee shed 100 paise to close at 87.52 per dollar on August 1 against the July 25 closing of 86.52.\', \'content:\\n\', \'heading: Chartist Talk: Sudeep Shah shares 2 buy and 2 sell picks amid market caution; Nifty at risk below 24,400\', \'content:The 24,400–24,350 zone is expected to act as an immediate support for Nifty. A sustained break below this could further accelerate the downside. On the flip side, the 50-day EMA zone of 24,900–24,950 now stands as a crucial hurdle, and any meaningful recovery would need to clear this zone with conviction, said Sudeep Shah of SBI Securities.\', \'content:\\n\', \'heading: Bulk deals: SR Foundation buys half a percent stake in Belrise Industries\', \'content:Belrise Industries shares has seen buying interest for fourth consecutive week, rising nearly 37 percent during the period.\', \'content:\\n\', \'heading: Top gainers & losers: Stocks that moved the most on August 1\', \'content:\', \'content:\\n\', \'heading: Business in the Week Ahead (August 4-8, 2025)\', \'content:Be it trade or monetary policy, the US reset is making many go back to the drawing board. What should be your market toolkit for the coming week? What are the markers you just can’t avoid? There are lots, apart from a couple of key credit policy calls, and the earnings dashboard. Hop on\', \'content:\\n\', \'heading: Taking Stock: Nifty below 24,600 in fifth week of fall, big selloff in pharma\', "content:Tracking weak global cues, the indices had opened lower but recovered, however, a mid-session profit booking and extended selloff in the final hour dragged Nifty below 24,550, ending near day\'s low.", \'content:\\n\', \'heading: Symphony Approves FY25 Financials, Declares ₹8/Share Final Dividend\', \'content:Symphony Approves FY25 Financials, Declares ₹8/Share Final Dividend\', \'content:\\n\', \'heading: Markets slide for second session amid global weakness, tariff Concerns\', \'content:Asian markets were largely in the red, with indices in South Korea, Japan, China and Hong Kong trading lower. The negative trend followed weak closing on Wall Street, and US futures indicating a subdued start, further dampening investor confidence.\', \'content:\\n\', \'heading: Textile stocks extend losses after Trump tariffs; Gokaldas Exports, Pearl Global, KPR Mill down up to 6%\', \'content:Textile stocks: Bangladesh has secured a 20 percent tariff deal with US. Indian textile exports will now face higher tariffs on exports than their rivals from Bangladesh.\', \'content:\\n\', \'heading: Live: Nifty ends fifth week lower, slips below 24,600-mark | Closing Bell\', "content:Nifty Extends Losing Streak To Fifth Week. Nifty Slips Below 24,700; Sensex Down 400 Pts. Barring FMCG, All Sectors Down In The Red. FMCG Best Sectoral Performer This Week. Realty, Bank Worst Performer This Week. Pharma Sees Severe Cuts On Trump\'s Letters. Broader Mkts Underperform; India Vix Up 2%. Nifty 50 Gainers: Trent, Asian Paints, Nestle. Nifty 50 Laggards: Sun Pharma, Dr Reddy\'s, Cipla. Join Lovisha Darad and Yatin Mota as they break down the week’s market action and highlight stocks to watch.", \'content:\\n\', \'heading: ABB India share price falls 2.01%\', "content:With the stock price at Rs 5,399.50, ABB India\'s performance reflects the current market dynamics and investor reactions within the NIFTY NEXT 50 landscape.", \'content:\\n\', \'heading: Bharti Airtel share price falls 1.39%\', "content:With the stock\'s last traded price at Rs 1,887.70, Bharti Airtel has seen a decrease of 1.39% in today\'s session.", \'content:\\n\', \'heading: Retail Losses, Jane Street & Weekly Options: What Needs Fixing in F&O?\', \'content:With retail investors suffering massive losses and fresh allegations of market manipulation involving Jane Street, the need to reform India’s derivatives market has never been more urgent. The challenge lies in striking the right balance—ensuring the market stays vibrant and efficient, without turning into a playground for speculation and manipulation.\\nMoneycontrol’s panel discussion on reforming the F&O segment brought together four seasoned experts to explore actionable solutions—from curbing excessive retail participation to rethinking weekly options.\', \'content:\\n\', \'heading: Adani Power announces its first-ever stock split: What it means for shareholders?\', \'content:Adani Power announces its first-ever stock split in the ratio of 1:5.\', \'content:\\n\', \'heading: BSE-listed smallcap declares final dividend for FY25\', \'content:The company previously set the record date for dividend eligibility on July 04, 2025.\', \'content:\\n\', "heading: ITC share price gain 1.27% in friday\'s trading session", "content:With the stock\'s last traded price at Rs 417.20, ITC has seen a gain of 1.27% from the previous close.", \'content:\\n\', \'heading: Smallcap NBFC to consider funds raising via private placement\', \'content:Smallcap NBFC to consider funds raising via private placement\', \'content:\\n\', \'heading: Sensex down 350 pts, Nifty below 24,650 as US unleashes tariffs; pharma, IT, metal stocks tumble\', "content:Laggards on the Nifty included Dr Reddy\'s, Sun Pharma, Cipla, Tata Steel, and ONGC.", \'content:\\n\', "heading: Adani Power Q1 results: Net profit falls 13.5% to Rs 3,385 crore, firm\'s board OKs 1:5 stock split", \'content:Adani Power Q1 results: Revenue rises 6% to Rs 14,109 crore\', \'content:\\n\', \'heading: Buy Transport Corporation of India; target of Rs 1350: Sharekhan\', \'content:Sharekhan is bullish on Transport Corporation of India has recommended buy rating on the stock with a target price of Rs 1350 in its research report dated July 30, 2025.\', \'content:\\n\', \'heading: Buy Amber Enterprises India; target of Rs 9300: Sharekhan\', \'content:Sharekhan is bullish on Amber Enterprises India has recommended buy rating on the stock with a target price of Rs 9300 in its research report dated July 30, 2025.\', \'content:\\n\', \'heading: Buy Bharat Electronics; target of Rs 445: Sharekhan\', \'content:Sharekhan is bullish on Bharat Electronics has recommended buy rating on the stock with a target price of Rs 445 in its research report dated July 30, 2025.\', \'content:\\n\', \'heading: Sell Mahanagar Gas; target of Rs 1266: Geojit Financial Services\', \'content:Geojit Financial Services is bearish on Mahanagar Gas has recommended sell rating on the stock with a target price of Rs 1266 in its research report dated July 31, 2025.\', \'content:\\n\', \'heading:Watch\', \'heading:You got 30 Day’s Trial of \']', name='get_topic_news', id='7fbb5a32-b72d-4a49-b070-3f6d83cec54f', tool_call_id='call_BCj4zYpikOre51Qgg29tdRVK'), AIMessage(content="**Timestamp: October 28, 2023**\n\n### Banking and Financial Sector Updates:\n\n- **Federal Bank**: Plans to grow its corporate loan book by 8-10% in FY26 amidst tariff uncertainties. Reported a 15% decline in net profit to Rs 862 crore for Q1FY26.\n- **IDBI Bank**: Financial bids for its stake are expected by the December quarter, with a winning bid likely by the end of the fiscal year.\n- **ICICI Lombard**: Scaled down its motor and fire insurance coverage due to pricing pressures.\n- **DCB Bank**: Achieved a 20% increase in net profit to Rs 157 crore in Q1FY26, with total income rising by 25.57%.\n- **RBI's MuleHunter AI**: Expanded its fraud detection tool to over 15 more banks.\n- **Rupee Performance**: The rupee opened 2 paise higher at 87.58/USD but is under pressure with a year-to-date depreciation of 3.25%.\n\n### Company and Market Updates:\n\n- **Tata Motors**: Considering a $4.5 billion acquisition of Italian truck maker Iveco, aiming to expand its global commercial vehicle business.\n- **Reliance Jio**: Raised nearly Rs 15,000 crore through a warrant allocation to fund its ventures in financial services and reinsurance with Allianz.\n- **KPIT Technologies**: Reported Q1 revenue growth of 12.8% YoY and strong deal wins worth $241 million.\n- **Adani Power**: Announced a 1:5 stock split and reported a 13.5% fall in net profit to Rs 3,385 crore for Q1.\n- **Asian Paints**: Experienced a 6% decline in net profit for the June quarter but saw a 59% sequential increase in PAT.\n\n### Stock Market Movements:\n\n- **Sensex and Nifty**: Both indices have been under pressure, with the Sensex down 585 points and Nifty below 24,600 amidst weak global cues and tariff concerns.\n- **Nifty Realty Index**: Fell nearly 4% due to unmet high expectations despite earnings growth.\n- **TCS**: Announced plans for a major layoff of 12,000 employees, raising concerns about AI-led job cuts in the IT sector.\n\n### Investment Insights:\n\n- **Maharashtra Temples and Trusts**: Now permitted to invest up to 50% of their corpus in mutual funds and market-linked securities.\n- **Adani Power**: Conducts its first stock split, aiming to increase liquidity and shareholder base.\n- **Gokaldas Exports and Other Textile Stocks**: Facing losses due to new tariffs from the US, which favor Bangladeshi competitors.\n\nThese updates highlight critical financial developments and market trends impacting various sectors, offering insights for investors and market participants.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 600, 'prompt_tokens': 25041, 'total_tokens': 25641, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 24960}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'finish_reason': 'stop', 'logprobs': None}, id='run--e2cca9dd-520d-4b9b-ba43-5ff163ba2280-0', usage_metadata={'input_tokens': 25041, 'output_tokens': 600, 'total_tokens': 25641, 'input_token_details': {'audio': 0, 'cache_read': 24960}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [13]:
for msg in response["messages"]:
    print(msg.pretty_print())

================================ Human Message =================================

user,
As per the current news tell me some good sector and stocks i can invest  with reason 

None
================================ Human Message =================================

You are a financial news summarizer working with web-scraped content from known news websites (e.g., Moneycontrol, Business Today).
None
================================== Ai Message ==================================
Tool Calls:
  get_topic_news (call_uY16WyUEWuoyEpZOCxhg9UQs)
 Call ID: call_uY16WyUEWuoyEpZOCxhg9UQs
  Args:
None
================================= Tool Message =================================
Name: get_topic_news

['new_source_bankingnews', 'heading: Federal Bank’s corporate loan book to grow 8-10% in FY26 on tariffs uncertainty, cheaper funds at bond market: CEO', 'content:According to the investor presentation of the bank, corporate loan book of the bank increased by 6 percent on-year to Rs 83,680 crore in Q1